The network is a feed-forward, fully connected neural network with six hidden layers with soft plus activation functions, the first three having 128 neurons and the last three having 64 neurons. For each formula, we use 80% as the training set and the remainder as the validation set, training for 100 epochs with learning rate 0.005 and batch size 2048. We use the rms error loss function and the Adam optimizer with a weight decay of 10^−2. The learning rate and momentum schedules were implemented as described in ***L. N. Smith, N. Topin, Super-convergence: Very fast training of residual networks using large learning rates (2018); https://openreview.net/forum?id=H1A5ztj3b*** and ***L. N. Smith, A disciplined approach to neural network hyper-parameters: Part 1 – learning rate, batch size, momentum, and weight decay. arXiv:1803.09820 (2018).*** using the FastAI package ***J. Howard et al., Fastai, https://github.com/fastai/fastai (2018).***, with a ratio of 20 between the maximum and minimum learning rates, and using 10% of the iterations for the last part of the training cycle. For the momentum, the maximum β1 value was 0.95 and the minimum 0.85, while β2 = 0.99.


In [5]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import numpy as np
import torch
from torch.utils import data
import pickle
from matplotlib import pyplot as plt
import torch.utils.data as utils
import time
import os

bs = 2048
wd = 1e-2

is_cuda = torch.cuda.is_available()

In [6]:
class MultDataset(data.Dataset):
    def __init__(self, factors, product):
        'Initialization'
        self.factors = factors
        self.product = product

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.product)

    def __getitem__(self, index):
        # Load data and get label
        x = self.factors[index]
        y = self.product[index]

        return x, y

def rmse_loss(pred, targ):
    denom = targ**2
    denom = torch.sqrt(denom.sum()/len(denom))
    return torch.sqrt(F.mse_loss(pred, targ))/denom

def NN_train(data, epochs=1000, lrs=1e-2, N_red_lr=4, pretrained_path=""):
    try:
        os.mkdir("results/NN_trained_models/")
    except:
        pass

    try:
        n_variables = len(data[0])-1
        variables=data[:,:-1]
        f_dependent = data[:,-1]
        f_dependent = np.reshape(f_dependent,(len(f_dependent),1))

        epochs = 200*n_variables
        if len(data)<5000:
            epochs = epochs*3

        if n_variables==0 or n_variables==1:
            return 0




        factors = torch.from_numpy(variables)
        if is_cuda:
            factors = factors.cuda()
        else:
            factors = factors
        factors = factors.float()

        product = torch.from_numpy(f_dependent)
        if is_cuda:
            product = product.cuda()
        else:
            product = product
        product = product.float()

        class SimpleNet(nn.Module):
            def __init__(self, ni):
                super().__init__()
                self.linear1 = nn.Linear(ni, 128)
                self.linear2 = nn.Linear(128, 128)
                self.linear3 = nn.Linear(128, 64)
                self.linear4 = nn.Linear(64,64)
                self.linear5 = nn.Linear(64,1)
            
            def forward(self, x):
                x = F.tanh(self.linear1(x))
                x = F.tanh(self.linear2(x))
                x = F.tanh(self.linear3(x))
                x = F.tanh(self.linear4(x))
                x = self.linear5(x)
                return x

        my_dataset = utils.TensorDataset(factors,product) # create your datset
        my_dataloader = utils.DataLoader(my_dataset, batch_size=bs, shuffle=True) # create your dataloader

        if is_cuda:
            model_feynman = SimpleNet(n_variables).cuda()
        else:
            model_feynman = SimpleNet(n_variables)

        if pretrained_path!="":
            model_feynman.load_state_dict(torch.load(pretrained_path))

        check_es_loss = 10000

        for i_i in range(N_red_lr):
            optimizer_feynman = optim.Adam(model_feynman.parameters(), lr = lrs)
            for epoch in range(epochs):
                model_feynman.train()
                for i, data in enumerate(my_dataloader):
                    optimizer_feynman.zero_grad()
                
                    if is_cuda:
                        fct = data[0].float().cuda()
                        prd = data[1].float().cuda()
                    else:
                        fct = data[0].float()
                        prd = data[1].float()
                    
                    loss = rmse_loss(model_feynman(fct),prd)
                    loss.backward()
                    optimizer_feynman.step()
                
                '''
                # Early stopping
                if epoch%20==0 and epoch>0:
                    if check_es_loss < loss:
                        break
                    else:
                        torch.save(model_feynman.state_dict(), "results/NN_trained_models/models/" + filename + ".h5")
                        check_es_loss = loss
                if epoch==0:
                    if check_es_loss < loss:
                        torch.save(model_feynman.state_dict(), "results/NN_trained_models/models/" + filename + ".h5")
                        check_es_loss = loss
                '''
                torch.save(model_feynman.state_dict(), "weights.h5")   
            lrs = lrs/10

        return model_feynman

    except NameError:
        print("Error in file")
        raise

In [7]:
def evaluate_derivatives(pathdir,filename,model):
    try:
        data = np.loadtxt(pathdir+filename)[:,0:-1]
        pts = np.loadtxt(pathdir+filename)[:,0:-1]
        pts = torch.tensor(pts)
        pts = pts.clone().detach()
        is_cuda = torch.cuda.is_available()
        grad_weights = torch.ones(pts.shape[0], 1)
        if is_cuda:
            pts = pts.float().cuda()
            model = model.cuda()
            grad_weights = grad_weights.cuda()

        pts.requires_grad_(True)
        outs = model(pts)
        grad = torch.autograd.grad(outs, pts, grad_outputs=grad_weights, create_graph=True)[0]
        save_grads = grad.detach().data.cpu().numpy()
        save_data = np.column_stack((data,save_grads))
        np.savetxt("results/gradients_comp_%s.txt" %filename,save_data)
        return 1
    except:
        return 0

In [8]:
class MultDataset(data.Dataset):
    def __init__(self, factors, product):
        'Initialization'
        self.factors = factors
        self.product = product

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.product)

    def __getitem__(self, index):
        # Load data and get label
        x = self.factors[index]
        y = self.product[index]

        return x, y

def rmse_loss(pred, targ):
    denom = targ**2
    denom = torch.sqrt(denom.sum()/len(denom))

    return torch.sqrt(F.mse_loss(pred, targ))/denom


def NN_eval(data):
    try:
        n_variables = len(data[0])-1
        variables=data[:,:-1]
        f_dependent = data[:,-1]
        f_dependent = np.reshape(f_dependent,(len(f_dependent),1))


        if n_variables==0:
            return 0
        elif n_variables==1:
            variables = np.reshape(variables,(len(variables),1))
        else:
          variables=variables



        factors = torch.from_numpy(variables[0:int(5*len(variables)/6)])
        if is_cuda:
            factors = factors.cuda()
        else:
            factors = factors
        factors = factors.float()
        product = torch.from_numpy(f_dependent[0:int(5*len(f_dependent)/6)])
        if is_cuda:
            product = product.cuda()
        else:
            product = product
        product = product.float()

        factors_val = torch.from_numpy(variables[int(5*len(variables)/6):int(len(variables))])
        if is_cuda:
            factors_val = factors_val.cuda()
        else:
            factors_val = factors_val
        factors_val = factors_val.float()
        product_val = torch.from_numpy(f_dependent[int(5*len(variables)/6):int(len(variables))])      
        if is_cuda:
            product_val = product_val.cuda()
        else:
            product_val = product_val
        product_val = product_val.float()

        class SimpleNet(nn.Module):
            def __init__(self, ni):
                super().__init__()
                self.linear1 = nn.Linear(ni, 128)
                self.linear2 = nn.Linear(128, 128)
                self.linear3 = nn.Linear(128, 64)
                self.linear4 = nn.Linear(64,64)
                self.linear5 = nn.Linear(64,1)

            def forward(self, x):
                x = F.tanh(self.linear1(x))
                x = F.tanh(self.linear2(x))
                x = F.tanh(self.linear3(x))
                x = F.tanh(self.linear4(x))
                x = self.linear5(x)
                return x

        if is_cuda:
            model = SimpleNet(n_variables).cuda()
        else:
            model = SimpleNet(n_variables)
                    
        model.load_state_dict(torch.load("weights.h5"))
        model.eval()
        return(rmse_loss(model(factors_val),product_val),model)

    except Exception as e:
        print(e)
        return (100,0)

In [12]:
variables=sample_data[:,:-1]
f_dependent=sample_data[:,-1]
f_dependent = np.reshape(f_dependent,(len(f_dependent),1))

torch.from_numpy(f_dependent[int(5*len(variables)/6):int(len(variables))])

tensor([[1.1829e+00],
        [6.7197e-01],
        [6.0110e+00],
        [1.8878e-02],
        [7.5423e+00],
        [5.3357e+00],
        [2.0178e-02],
        [1.7140e+01],
        [4.2290e-01],
        [1.2401e+01],
        [9.1837e+00],
        [2.0489e-01],
        [3.0428e+00],
        [1.6286e+01],
        [1.5341e+00],
        [6.6775e+00],
        [5.8065e+00],
        [8.3367e+00],
        [1.1916e+01],
        [8.6685e-01],
        [7.1504e+00],
        [1.6417e+00],
        [6.0569e-01],
        [2.2456e-01],
        [1.5270e+00],
        [4.9932e+00],
        [7.1552e-01],
        [7.7674e+00],
        [1.1213e+01],
        [9.5092e+00],
        [1.8795e-02],
        [2.5412e+00],
        [4.5042e-02],
        [1.2082e+00],
        [7.4210e-01],
        [9.3957e+00],
        [8.9278e+00],
        [3.8571e+00],
        [1.4308e+00],
        [6.1559e-01],
        [1.7259e+00],
        [1.7519e-01],
        [4.0949e-02],
        [2.9215e+00],
        [2.1328e+00],
        [1

In [42]:
NN_eval(sample_data)

(tensor(0.0008, grad_fn=<DivBackward0>),
 SimpleNet(
   (linear1): Linear(in_features=3, out_features=128, bias=True)
   (linear2): Linear(in_features=128, out_features=128, bias=True)
   (linear3): Linear(in_features=128, out_features=64, bias=True)
   (linear4): Linear(in_features=64, out_features=64, bias=True)
   (linear5): Linear(in_features=64, out_features=1, bias=True)
 ))

In [9]:
sample_data=np.array([[3.6756507480135925,
  4.3458834950352285,
  1.7605189339905305,
  12.284226801691412],
 [3.7073328902707914,
  1.1346581008306025,
  1.4329823253244038,
  0.1649713882766933],
 [3.2291698349716906,
  3.5637395108734933,
  2.5961086203758703,
  1.5117512617780429],
 [4.222577473924653, 2.0059470527747085, 4.67452731302258, 15.035163986446232],
 [2.863786825709689, 2.755691724421186, 4.344721344604347, 3.6155525374748505],
 [3.054178670722203, 2.342059777944911, 1.6273255615200917, 0.780105951718621],
 [1.1864909079293597, 4.388505107807092, 3.518003384103658, 0.449545536274374],
 [2.233852805108659, 3.7516598703045436, 1.14593811304582, 7.583691112875774],
 [1.8882358103854018,
  2.133246983399676,
  1.6788981597630772,
  0.19489695324709982],
 [4.006083064733118, 1.297092706236067, 4.356751789899231, 18.751500763428634],
 [1.3808571059075527, 1.2569416495052796, 4.75471617432175, 8.446997471922536],
 [1.9291865084649693,
  1.31958704497342,
  2.7044600350881747,
  1.8499675499869308],
 [4.301329764229804, 2.8333754548849046, 2.039263684116219, 1.356238318258302],
 [1.8459011108925751,
  1.1088259006204968,
  2.307639356492518,
  1.3264218075039622],
 [3.842015992840293,
  4.924736181176744,
  4.918755962580132,
  6.870103676001993e-05],
 [3.723643754611666,
  3.6945681241486277,
  3.9711182806427527,
  0.14239211680229377],
 [4.234921738757024,
  3.6588917555385243,
  2.4443848150660554,
  3.123312183427685],
 [3.5313391108890237, 4.28845313582521, 4.635074770867886, 0.2121391194395643],
 [3.8319100916351245,
  1.08859070925757,
  1.9581750943182672,
  1.4488011439335686],
 [1.0232693619602395,
  4.320360207864704,
  4.420133702289746,
  0.005093195437586879],
 [1.7710253381875107,
  4.455363408995632,
  2.5963759664511126,
  3.0601850651559843],
 [2.5963138612377596,
  2.7379932620017335,
  2.129841004371465,
  0.48012226132100766],
 [3.2259567741610713, 4.24575973029801, 3.628955044500185, 0.6136544343485087],
 [2.3203520808980294,
  3.5993632169644783,
  1.7109304245978874,
  4.137394748739946],
 [3.1691240642361973,
  2.96542526194728,
  3.2154214203715767,
  0.09903208342326417],
 [2.1889248265935786,
  1.9894464467347062,
  3.7756585503679614,
  3.4919410794529036],
 [2.4666872815891043,
  3.84668087308085,
  4.5948961758449585,
  0.6904580088443635],
 [4.4745465960272615,
  3.5764875765551682,
  3.979142443646399,
  0.36273122730107304],
 [3.1145701893935325,
  1.698575266804577,
  3.0492765014440155,
  2.8411013109287695],
 [2.1189702052931523,
  1.5225259216529188,
  3.418639526883071,
  3.8091104291109654],
 [4.819772550339692, 4.7098982741836295, 3.594957069962799, 2.995714901533866],
 [2.387230822838141, 1.0040048443229983, 3.444569661624295, 7.109599066431139],
 [4.58859173460244,
  3.2751706222505845,
  3.4578839389460088,
  0.07659313136861863],
 [4.936818618148065,
  3.7282411113683938,
  2.6861260329717584,
  2.6807019800069347],
 [1.7862640335164364,
  4.362121428629974,
  1.8573940492926417,
  5.603205933763905],
 [1.1336376009746147,
  4.467168779171779,
  4.9636241334607085,
  0.13970265009403596],
 [4.913451344596778, 3.781021029957085, 1.984808235667714, 7.92633156309991],
 [4.0237166829032, 4.401731643505678, 2.923923818508433, 4.393729556491852],
 [4.582070783305149,
  3.614167824843202,
  2.8686056250184895,
  1.2735017917228566],
 [1.143337730670405, 2.3086476753262004, 4.21315690494305, 2.0735318151685442],
 [4.332289461421704, 1.0174223922197414, 4.313802316477983, 23.53758989190368],
 [1.8693469674579135,
  1.931305057578752,
  1.8506633087282158,
  0.006078267334685901],
 [4.31825988311849,
  4.2007615117923915,
  4.059411618475764,
  0.043138967868773116],
 [2.6325946385526238,
  3.0328756500119582,
  1.89993827910755,
  1.6895295889824373],
 [3.407172731519162,
  2.4582774169700006,
  2.3258539684563706,
  0.029874038918962793],
 [1.8320274962667473,
  1.0849166587734973,
  2.0242205750222104,
  0.8081914617914234],
 [4.872410359419993,
  2.7882356654789313,
  3.805259187864362,
  2.5198567795616262],
 [2.139206780495007,
  2.846923585980025,
  3.5994373321700586,
  0.6056917329229996],
 [2.624277108766037,
  4.208645358873247,
  4.104905159565406,
  0.014121273111870582],
 [3.2180657650505475,
  1.9276636460254974,
  4.649722286505903,
  11.922295263188262],
 [1.5847591418557658,
  3.676699211134713,
  4.978293743358334,
  1.3424085238932162],
 [2.4690126349398946,
  1.7961823151497454,
  1.6926375575034784,
  0.013235780266932217],
 [1.8000876238590156,
  3.1348504111746367,
  4.521323855995645,
  1.7301619719239032],
 [4.21039034198468,
  2.1990136128206283,
  1.5269889168839743,
  0.9507423316276341],
 [3.831326861350709, 2.636920779140794, 4.002814024567709, 3.5739849843489435],
 [3.4148875335194857,
  2.432264867286026,
  1.6287406099899422,
  1.1024131716870988],
 [4.784827278442153,
  1.2286146457770988,
  3.5356203773913646,
  12.733084367986683],
 [3.3169528952136638,
  4.160273676713628,
  4.126188164509452,
  0.0019268546591817068],
 [4.7096694422506715,
  4.186610780156741,
  2.005126551984683,
  11.206360404686436],
 [3.0000494716071078,
  1.4688402154931683,
  3.4006766372702915,
  5.598080254576958],
 [4.566634260262634,
  1.8423615930829333,
  4.3406249586608965,
  14.250912513951784],
 [1.5103234254964102,
  1.4479890278097036,
  2.470929220592424,
  0.790206228990986],
 [4.307754756903698, 4.229204034805557, 1.5971280075925693, 14.9216838543348],
 [1.8067379638598466,
  3.5465266435673404,
  2.4836950932933908,
  1.020455452556243],
 [1.9503482409384616, 2.928624911485564, 4.983854188340912, 4.11910367511457],
 [1.3502019895578385,
  4.706683969418755,
  1.6273124959193361,
  6.401664539353142],
 [1.7163421430803583,
  3.3352654749130997,
  3.9120181901950306,
  0.28546519582316227],
 [2.159477427995817,
  2.6935426112587657,
  2.481091958234183,
  0.04873430915136065],
 [4.516355343937445,
  4.6592295700132365,
  1.3297671474059425,
  25.032622163817503],
 [3.0881322586594133,
  4.805160281973958,
  1.8977790872371494,
  13.051783177694016],
 [2.3669788678938692,
  4.219936023518497,
  4.270123321412427,
  0.0029809316101163785],
 [2.360646526434459,
  2.6213728345270226,
  2.6900410320713646,
  0.005565603487657909],
 [2.365872957208551, 4.917269032751785, 4.057708332564967, 0.8740062261725429],
 [3.9223364527834965,
  4.852834069198913,
  1.6395848851277224,
  20.249003727931573],
 [3.105648801575125, 2.177745697968549, 3.5886812434956634, 3.091268271249812],
 [4.195762706709374,
  1.961203174992865,
  1.5159543255912182,
  0.41589771521387653],
 [4.418558467323633, 4.453922614804991, 2.94471120267864, 5.032117477902857],
 [3.909422065642121, 4.297573226089108, 2.8917565290947644, 3.863135652962223],
 [3.8754045884724087,
  2.3062433157525115,
  3.9073012748597784,
  4.967080073397159],
 [2.8441444954758848,
  3.862754128393849,
  4.023928324871109,
  0.03694134371782322],
 [3.6666526576919325,
  4.597300181024147,
  2.9363194501522707,
  5.057885154221813],
 [2.4990541356507965, 1.227134819593969, 3.229841977687109, 5.011648097973834],
 [2.395624698999885,
  1.1005169410527835,
  4.476314623621494,
  13.650281505402294],
 [3.81772107208474, 3.836641846376655, 2.202282686041986, 5.09881438588512],
 [2.489589243637452,
  3.6745160174114346,
  3.8796409607582576,
  0.052376280224746004],
 [1.7713832330045753,
  4.450942250351103,
  3.662682634357882,
  0.5503273397943426],
 [4.285965176877468,
  3.307911433674372,
  3.8550153427092666,
  0.6414433071684376],
 [2.877143708282607, 2.3035285899425637, 4.29797040453052, 5.722348462601542],
 [1.7156267219614434,
  3.044347051193655,
  4.2029612322481285,
  1.1515173502640959],
 [1.8041946793105215,
  2.3873917581054007,
  3.9227427777416732,
  2.126516542692695],
 [1.9063672632416147,
  2.0961513203514532,
  4.687622319607055,
  6.401316227708364],
 [2.971469188896023,
  2.2762466664132535,
  2.3364262911659335,
  0.00538071744240556],
 [3.6922060576710467,
  3.0313639927792613,
  1.4557767532297792,
  4.582904892373778],
 [1.0441568702215451,
  4.476249812987317,
  4.740906566053985,
  0.03656804265057024],
 [3.0843866203174866,
  4.44136830895411,
  1.1140721956873105,
  17.073467031231008],
 [3.3209786057482136,
  2.6748244584952983,
  2.810158363827785,
  0.030412303160281418],
 [4.026896431514748, 2.3710876956850124, 4.246720465664099, 7.083307375647512],
 [4.38144400205083, 4.0356708324167165, 3.636218356236789, 0.3495565989208456],
 [3.230805389553694, 3.895019679189576, 4.655786750901584, 0.9349410241646644],
 [2.5862946780883544,
  2.434394079743094,
  1.8382414397851563,
  0.4595819393737837],
 [4.260404266522695,
  4.608650653424267,
  1.7394205293818672,
  17.536849663372863],
 [3.968572085408236, 2.883653137980493, 1.813887029132161, 2.2708160099742325],
 [3.931058553932684,
  1.3557813067768505,
  1.4604638685492324,
  0.02153913217179584],
 [2.6557030486092215,
  3.1898202188717883,
  3.5031288469032495,
  0.1303449549025954],
 [4.213838302600291,
  4.005323587993255,
  4.161637877828285,
  0.051480793764527175],
 [4.833664097555117, 2.3087979918758244, 3.506333334136134, 3.465956838222334],
 [4.55065144947896,
  3.690060400020224,
  3.5015596718374904,
  0.08084806711762643],
 [3.8909838218294324,
  3.7010913767937277,
  3.197397098246254,
  0.49358671823770817],
 [2.118464854692712, 2.030138622970704, 4.3964079573973, 5.930886580568904],
 [3.769022101279546,
  3.4923359751349157,
  2.052424272535769,
  3.9072429047074606],
 [1.1297670437769791,
  3.779462405182012,
  1.0868494476011974,
  4.095498479248956],
 [2.8097698959788775,
  1.9100281130392451,
  4.084999069048895,
  6.645806363195441],
 [4.9729811514192654,
  3.595668850403789,
  3.2781986740995097,
  0.25060670353439785],
 [2.0565501959386445, 1.102957172390306, 3.2444574710696084, 4.71569379382252],
 [3.264955918719926,
  1.5264259380947598,
  2.4533735419559592,
  1.4026770739261698],
 [2.6836568871783055,
  1.150064200483663,
  4.275703604981467,
  13.109156262654126],
 [1.8692872476203797, 4.771816287407914, 1.705813628560136, 8.785998035433117],
 [3.5659522498997713,
  1.3217353077606888,
  3.9733255165002737,
  12.53598145853286],
 [1.3813562095489345,
  4.417350997762995,
  3.1901297621690565,
  1.0402109277414797],
 [4.072970806005928,
  4.539537002443529,
  1.9456846797890561,
  13.701616084182545],
 [1.6244484388756177,
  1.9589630499470863,
  3.135833587914791,
  1.1249502510283846],
 [3.6707600008542833, 4.418833848972855, 3.519937221860672, 1.483014839439469],
 [2.3353400920135154,
  3.6170077891385937,
  2.9185570772401572,
  0.5696284450619123],
 [1.433541937205947, 1.3587122886269678, 4.691372912733462, 7.960907674771527],
 [3.9270020870004063,
  3.984583118649289,
  3.6083432343348045,
  0.2779462383671003],
 [1.4116775929004972, 4.044271711912306, 2.82335482273078, 1.0521501673587175],
 [1.3036285803264627,
  4.396941527036615,
  4.712083375323333,
  0.0647345350629608],
 [3.35411228368542, 1.8924096532044472, 4.301824692474733, 9.735781873524825],
 [1.4718649342422925,
  4.849159310982441,
  4.300864895096967,
  0.22124099794651642],
 [2.0163066242697156, 1.977152084519429, 4.0468239561581285, 4.31846670841853],
 [1.319569676218936,
  4.9321135851696685,
  2.0886286291367044,
  5.334628747432363],
 [4.343463934648858,
  2.187583386154123,
  2.1725498477041967,
  0.0004908272311872862],
 [3.3626804482069788, 3.830359114286515, 2.012658204262402, 5.555209634636036],
 [4.253802440118299,
  2.882851845925183,
  2.2954426967772856,
  0.7338862206151101],
 [3.2004592542824453, 1.147811939023475, 4.29051268018255, 15.80477664563398],
 [4.877186160174558, 3.077270842932459, 2.1484374533664417, 2.103850981919135],
 [3.007774359007106,
  2.116268366595686,
  2.8974048409051925,
  0.9176331438558123],
 [1.4307026897696469,
  2.277066385430533,
  2.397656307920182,
  0.010402588757844854],
 [2.014957672291816,
  3.1562114437614253,
  2.2380817479207664,
  0.8492665140946424],
 [1.1489392701490866,
  3.535850100852743,
  1.4701406514239777,
  2.4513512798147996],
 [1.9774946313134092,
  4.964631993624375,
  3.3664419827038654,
  2.525469577377271],
 [4.563797253122699, 1.071682373230106, 4.805677079220109, 31.81586555056052],
 [2.004393245188922,
  2.992508433723464,
  3.0800176591270207,
  0.00767468596898399],
 [1.9503288515719643,
  2.1494758525065283,
  1.4053976780109094,
  0.5399020562361961],
 [4.9486991344445945,
  1.5830196808412014,
  3.88741142049472,
  13.139343750230111],
 [4.75747780526862, 4.398488401063261, 2.2523379493803284, 10.956380425552487],
 [4.449357663413542,
  4.563025648046622,
  2.0401620434462093,
  14.159726522457744],
 [3.26716379047544,
  3.0559639184568397,
  2.4131638065112493,
  0.6749829441845968],
 [3.6380207137225553,
  1.8056667401037334,
  2.7761376074794946,
  1.7131688825830742],
 [2.2998739882960866,
  2.960900829907186,
  3.628154838741108,
  0.5119840471867506],
 [1.0206893757647189,
  3.0829149197762207,
  4.778623754962446,
  1.4674596367461903],
 [3.7847780419950134, 2.759046352200536, 4.937958240921388, 8.984414017856038],
 [2.9922658008373695,
  3.2288211536083757,
  1.3065857800570035,
  5.528194357235666],
 [3.3214930709998867,
  4.940472661520616,
  4.140886922031795,
  1.0617772969896417],
 [4.171893049438397,
  1.4846159940178811,
  1.8081049655086985,
  0.21828411328763395],
 [1.244498933608694,
  3.3877128111136203,
  2.795848392480135,
  0.21797615989980093],
 [1.6279198446796177,
  4.0514830550712615,
  4.940920912785236,
  0.6439234526010835],
 [2.359405788689709, 4.127389360359544, 2.143771761188482, 4.641822726951596],
 [4.331017424002915,
  4.358727134569774,
  1.1589087257314694,
  22.172292563950215],
 [2.618507390454929,
  4.9178794904777074,
  3.0811285507677004,
  4.416968984936604],
 [3.29213400231357, 2.367395201419945, 3.566586462527088, 2.3671425861202433],
 [1.7868716955872737,
  3.6449731913459646,
  3.7943583018480456,
  0.019937835077745828],
 [1.6139408956245909,
  1.7044893430360775,
  2.879812585270061,
  1.1147366491138224],
 [4.403038402753517,
  1.1182643627056965,
  1.639941375551195,
  0.5991366385629527],
 [1.5925044046482246,
  2.409840247058577,
  2.8176917211426282,
  0.13245084067741894],
 [2.909510805551838, 2.1799053091123244, 4.422164773518303, 7.314113752628628],
 [1.9939769256451667,
  2.3024627914308358,
  2.866054771595467,
  0.3166793477236085],
 [4.130311081272457,
  4.7568914345546816,
  2.033019130728677,
  15.322380907082177],
 [1.6515347388419332,
  1.0271573653501873,
  1.9610374188270159,
  0.7201781096753183],
 [3.5700669174328437,
  2.7930205138976145,
  1.2470155980303366,
  4.266464162518162],
 [1.125308171283022,
  4.733905601495595,
  2.7535728700828415,
  2.2065705018881974],
 [2.7345777208786193,
  1.8860592518888213,
  3.1867195006924502,
  2.313065922351824],
 [1.6793001200590054,
  1.7483725284412657,
  3.365070651229823,
  2.1946041264043594],
 [1.7288545890286469,
  4.078053229582243,
  3.394215982692686,
  0.40423505769982254],
 [4.9453201170213354, 3.000809077982299, 4.448077193811944, 5.179196666422978],
 [1.503474082622828,
  2.149144053254059,
  2.5169312154049677,
  0.10168551253832037],
 [3.5287644731494163,
  1.9858682281984485,
  4.775819112607929,
  13.733644216568454],
 [3.877152424601182, 1.964086932810459, 4.462711927526122, 12.1027772293475],
 [4.580853709354885, 1.01537267119107, 3.9470816779963944, 19.686030315158856],
 [4.342449677415532, 4.784742955685092, 4.053019165750236, 1.1625165621011715],
 [3.831964802058877,
  2.460242294159754,
  1.2256569181863486,
  2.9203423885177093],
 [3.5951701621254646, 1.999260591778774, 3.900577686422407, 6.498282101766773],
 [3.2652869972335457,
  3.836240958699032,
  4.329360471978905,
  0.3970047838783197],
 [3.1606390997065152,
  3.296752762697193,
  3.1505207731160874,
  0.03379322893590384],
 [1.7520895814330992,
  1.8719304781922421,
  1.5198652761283489,
  0.10858566990288661],
 [1.543774720703854, 2.034638016333449, 1.369970233448616, 0.3410069156479788],
 [2.773136535697741, 4.065554518871798, 4.848156507071202, 0.8492257431636046],
 [3.6016770306009653, 4.96986229934169, 3.2096456249137804, 5.579650958246009],
 [1.7535745514284753,
  1.2195019721118459,
  3.6676814363329884,
  5.255097037686241],
 [1.5907503309825732, 4.44032078879774, 2.5724397246589787, 2.775047782237577],
 [3.378912508970773,
  1.8052178617635959,
  4.849903666304995,
  15.661448104126508],
 [2.7080799048060498,
  3.848121333064344,
  1.4757455200694434,
  7.620762974200872],
 [3.454373257170154, 3.386333365143715, 4.58073377323389, 2.463991205187935],
 [4.425636075880046, 2.8918410023554055, 4.574927604941649, 6.268427814047749],
 [2.027338832239256,
  1.479905708857531,
  3.5481645010112324,
  4.3361680165354315],
 [2.5932812512892336,
  2.3385336863283976,
  1.550214549083444,
  0.8057935074668129],
 [1.6251268351030612,
  4.807670942760232,
  2.2128922661438453,
  5.470889041791078],
 [3.9918501207192763,
  3.129365175050542,
  2.6616372844400855,
  0.4366472872925612],
 [3.3781667407504123,
  2.4371971496345792,
  3.896547018300433,
  3.597244298224734],
 [4.564384309847281,
  3.869864731442692,
  1.5464939240523194,
  12.319391717754621],
 [4.126071287487287,
  4.056273840442956,
  4.262839043629992,
  0.08802804576439403],
 [4.600956321027031,
  3.4684018053023085,
  4.247436286700826,
  1.3961480564705708],
 [3.033315799571074,
  2.7052852386260606,
  3.3580987810345415,
  0.6463473042712639],
 [4.725338475188375, 3.1583619550173077, 4.163386138998931, 2.386469846969252],
 [2.1519991465522565,
  3.620497913136354,
  2.336060758307022,
  1.7751612898637317],
 [1.0618796090390186,
  3.115747906945961,
  3.0602154535578183,
  0.0016373405103756722],
 [3.096583151969257, 2.84059325089824, 3.3147973431890914, 0.3481635852851049],
 [1.8429598164480439,
  3.1824310068741894,
  3.8368552796745194,
  0.39464324048948846],
 [2.0279593381211005,
  2.9833795131974394,
  3.9432789346863775,
  0.9342878628981653],
 [4.395340333985407,
  4.665273691273564,
  4.2304624371277075,
  0.41549333865565646],
 [1.9147539239008666,
  1.741779915417626,
  1.447436279211121,
  0.08294539389560378],
 [2.0830354712535026,
  2.3512691084943262,
  3.9120132270510717,
  2.5370561776662637],
 [2.9469857447999552,
  3.07963717663667,
  2.2339543257553345,
  1.0538118725655852],
 [4.929765573909205, 3.6505961650862115, 4.66820715422622, 2.5524653107896143],
 [2.7813230918026948,
  3.756850388157447,
  3.007559672317394,
  0.7807682578795759],
 [1.8085428916994548,
  1.4331839543177298,
  1.173492127983403,
  0.06098392584297233],
 [1.5183960404679984,
  4.240051534609563,
  1.2177655543808936,
  6.9346760815374715],
 [2.784913246242138, 2.820297855998203, 4.561109798863617, 4.219737061507557],
 [1.261119308826808, 3.979666791535115, 1.7999163941330365, 2.995985523449463],
 [3.5315091205991815,
  4.5368851929499545,
  4.221944435372328,
  0.1751410996691275],
 [2.0451622006820025,
  1.6548559577965616,
  1.9159369039237695,
  0.06970246185403296],
 [2.292539301597736,
  1.6983066941131715,
  4.9409472380536315,
  12.052701783025551],
 [4.321575983564653, 2.062527807151433, 1.674978141381977, 0.3245389980496945],
 [2.0241811348765975, 4.70323446240643, 1.8785130683623574, 8.075522207622885],
 [4.750257618303155, 2.8903526681619276, 4.68078664270228, 7.613840733370807],
 [4.759895119536894,
  1.273407686392407,
  2.0457444455650298,
  1.4196484047633315],
 [3.584697260584405,
  3.1893182970515235,
  3.8254858458606007,
  0.7253798909519639],
 [1.8315586760736973, 1.5209926633661732, 4.326501084396947, 7.20798698715374],
 [3.8154878495977105,
  1.702460587802313,
  2.783853459463689,
  2.2309358587751227],
 [4.116408204439381,
  1.507398161378282,
  4.6838259023597555,
  20.766647921363404],
 [1.4024090569657544,
  3.8678686134556197,
  2.6744785735304784,
  0.9986413162934237],
 [4.18194083765766, 1.8665018419173323, 3.158514191305658, 3.4904483701142515],
 [1.8437344790809838,
  3.934648953311074,
  4.645685676346593,
  0.46607139009449094],
 [4.989867014554883, 1.347037330242295, 4.141660587492713, 19.48522897637527],
 [2.789754529426951, 4.85758048493421, 2.424049122611237, 8.260567625957641],
 [2.5171039897704808,
  1.793259263538351,
  1.0115801218629135,
  0.7690033100809254],
 [2.6666980075156865,
  2.012347463639045,
  3.604734578017058,
  3.3809686981602423],
 [4.545877996027327,
  1.4160510085964795,
  2.2326219980889936,
  1.5155688597385222],
 [4.27896182379264, 2.3639317444780064, 4.721125545477811, 11.887731755376459],
 [1.955887286440205, 2.9154613635862376, 1.823599650764924, 1.165867249515309],
 [1.9390417382812384,
  1.9928082348481957,
  3.9596504135321697,
  3.750560608702812],
 [2.293908585595183,
  2.576693411111175,
  4.2383088058056835,
  3.1667014846899963],
 [4.478608357265995,
  2.3874672612640686,
  2.6041008583893217,
  0.10509080352662169],
 [3.629005595039051,
  1.1834669916709397,
  3.6526144685503765,
  11.062459722558824],
 [2.27163497865638,
  1.5861621139407927,
  2.5521676355499285,
  1.0599070217218816],
 [3.7498946779906643,
  3.1231288616659696,
  1.4875726170371295,
  5.015567059504091],
 [1.7364533596043872,
  4.119214494926622,
  4.347545533898536,
  0.04526505296058805],
 [4.643914189343835, 4.250008471429922, 3.1405546785579967, 2.85806847075717],
 [2.9865612474357714,
  1.2746379933982044,
  2.995682674815681,
  4.423089435576881],
 [4.799038678700111,
  1.1863635339261331,
  3.0933874047904273,
  8.726428067877716],
 [4.739336648177832,
  1.8946678489689868,
  4.734180592673129,
  19.106239065974286],
 [2.28478593626799, 4.497795983507671, 1.265179575955016, 11.93778813530822],
 [1.5263349318841435,
  4.353773865718127,
  4.195295293686508,
  0.01916730027999706],
 [3.5255235059150682,
  4.805444096383544,
  3.7573869398758686,
  1.9362594690076491],
 [2.4919782457489017,
  4.1842774126042555,
  3.8459085502938404,
  0.14265763941859835],
 [1.6299395813459463,
  4.447840796506357,
  2.1521507260350714,
  4.295048004242472],
 [3.276223542784705, 4.917388822783327, 3.9786916105637666, 1.443426210939639],
 [1.3569559222611605,
  1.9022820092593666,
  2.3658134693879465,
  0.14577873445522302],
 [4.511288386110873, 2.1615023955509947, 4.052152756344448, 8.06293277010946],
 [1.2700021631647425,
  2.0414942114426404,
  4.174784234815708,
  2.8898431378293297],
 [2.537484900385465, 1.014984028374446, 3.9933482199860655, 11.25457434920563],
 [4.505649700611302, 4.326047282764446, 4.80174178844693, 0.5097810631507372],
 [3.1260284615860896,
  1.75861273598069,
  1.1987139419045914,
  0.4899841101308753],
 [4.275740533753547, 4.67696040345883, 2.795297712336976, 7.56945994483158],
 [4.916352386691198, 1.4280133170699267, 2.4095394002424295, 2.3681908484035],
 [1.859230954116883,
  4.360071430227073,
  3.7601031283784434,
  0.33462621216456173],
 [1.6885964510612514,
  3.8918133049594914,
  3.2086306418023125,
  0.39406653059743046],
 [4.146232389446838,
  2.0223643759639147,
  3.6530525162198653,
  5.512714098208758],
 [4.221292493972972,
  2.9837303186286994,
  3.0701174093872923,
  0.015751181905355238],
 [1.2321391338021295, 2.0692945245536905, 3.996753993394025, 2.28876005047214],
 [4.176796808180954, 3.180963688115717, 3.935260756454915, 1.188223650144816],
 [1.9052596226104477,
  3.619064135052037,
  2.2004822565732374,
  1.9170479839822976],
 [3.3112239131045866,
  1.3541163056045349,
  1.4015166517206072,
  0.003719817043316221],
 [4.267919145889666, 1.087150916086303, 2.4962307879262027, 4.236989717704716],
 [1.0165070872606012,
  3.1039688860086825,
  4.091516905818281,
  0.49567482314854444],
 [4.401032054710738,
  3.6530100715626563,
  4.742424022166173,
  2.6116224957009853],
 [4.8676649782188175,
  4.239236027164104,
  4.095609496331875,
  0.050206509081698096],
 [1.5694445096787635,
  4.677763256359846,
  4.06142547240255,
  0.29809421950475107],
 [3.867023950265398,
  2.2638595307460356,
  3.4870021501994937,
  2.892684472587615],
 [4.318433801761304,
  1.2036867741745563,
  1.649755803338306,
  0.4296357509961229],
 [1.1199017163474174,
  4.082269482038235,
  4.573106619263521,
  0.13490397405391347],
 [3.2247231098463764,
  1.6654401322665398,
  1.5944492016051015,
  0.008125838257487979],
 [3.638195315890749,
  4.294558414190993,
  1.4517132559265002,
  14.701526321161523],
 [3.917724199997822,
  3.7885049350448594,
  4.899921592441547,
  2.4196785056770005],
 [3.7099049264148576,
  1.0930077416757311,
  4.424812734821917,
  20.591687268070515],
 [2.5024413813715602,
  4.4344346812138475,
  4.70889312737191,
  0.09425124983713776],
 [4.034041838811103, 1.3556824022138012, 4.80481263926602, 23.995488141657727],
 [4.171159208320273, 1.698292465448326, 1.3586867064611, 0.24053421610370967],
 [4.819079240627056,
  4.2711534013593315,
  3.1954445913949603,
  2.7881974315441087],
 [3.879354227483484, 4.404663456229855, 3.068398541736612, 3.4634950611399185],
 [2.615229552411266,
  1.2693059797139514,
  1.4001660453652809,
  0.022392061961513735],
 [1.9618681023587183,
  4.4014791166866125,
  4.984000067690397,
  0.33286099739281905],
 [1.038644609519448,
  2.3698788522051877,
  2.289981703780994,
  0.0033151221453021833],
 [1.4141134005273739,
  2.1371467917448284,
  3.6999890792705004,
  1.7269690321188198],
 [2.227937817997283,
  3.084310040958472,
  3.3606504320210924,
  0.0850671348365706],
 [2.054253761269023, 2.0295486025075062, 4.843595024554725, 8.133671161237991],
 [3.017220073946857,
  3.1475098171840923,
  2.929400228989439,
  0.07176728358565562],
 [2.3127804065143325,
  2.594961915625112,
  1.4175800008631563,
  1.6030206789774206],
 [4.330810648184678,
  1.6606190093183142,
  2.0369595212779776,
  0.3066910786747497],
 [3.7921026256765735, 2.655570572187714, 4.837440673834648, 9.02626061601238],
 [1.9215799369694548,
  1.4658281614587754,
  1.3055098076957719,
  0.02469419932104072],
 [2.4775201477075037,
  3.099385132758661,
  3.5339561005841382,
  0.23394222589428557],
 [4.005987041091155,
  2.0071623990231813,
  2.103209351975335,
  0.018477649621222413],
 [3.9677956911911663,
  1.164620422166192,
  2.4280807573975247,
  3.166959652759875],
 [4.739902841887268,
  3.1970286907217007,
  3.639057849059072,
  0.46306427921347315],
 [3.660633002138485,
  1.1405246161964793,
  3.134952811009739,
  7.2805301585804765],
 [2.0464366874382582,
  2.1099986068099392,
  4.035574102132974,
  3.7939309147081866],
 [3.0562987594385045,
  3.962491342843453,
  3.8871801377573947,
  0.008667323438941214],
 [4.04791986208887,
  4.0309015178059235,
  1.0329463441705404,
  18.190815962446084],
 [1.2473744854734647,
  2.4891045327541885,
  3.2349634412794446,
  0.346960650540833],
 [1.0686141561014284,
  4.152433959817442,
  3.3995879514476646,
  0.30283302278881585],
 [3.90180625228959, 2.59037557413727, 4.075327582494506, 4.3019022785196555],
 [4.630365759422992,
  4.7128982758610345,
  4.558004580920766,
  0.055545998995569654],
 [1.3664260085293378,
  1.4509059750616689,
  3.2658384044390303,
  2.250489782678208],
 [4.365200517553733, 2.8577694890881684, 1.851984505259907, 2.207925916160999],
 [3.3310692912954925, 2.393191729182329, 4.334873854142956, 6.279281258176965],
 [1.7480623293313111,
  4.015201205716211,
  2.353197188060136,
  2.4142990128390283],
 [3.5961865495230962,
  1.4381187798921609,
  4.837254518437952,
  20.77539224490797],
 [2.9040504581116005,
  2.5277344326954103,
  2.8634710032776978,
  0.16367089688293954],
 [2.3022003980457044,
  1.7573504583039736,
  1.70713744346653,
  0.0029023228712757414],
 [4.625205835475631,
  3.897065306903961,
  3.9034976350595314,
  9.568358842643636e-05],
 [3.99798407334103,
  1.8916429538036548,
  1.7948813864844508,
  0.018716164460159054],
 [4.7033390625673945,
  4.265530977867545,
  1.725514806624211,
  15.172224338099664],
 [4.9525911208721825,
  4.482407574913793,
  2.6946474761530284,
  7.914453995327848],
 [2.9776412014370197,
  2.1493410106300654,
  4.412781888075125,
  7.627473005822095],
 [1.7766112076655496,
  4.481580525853948,
  3.611527810688899,
  0.6724399932966748],
 [1.374484848684094,
  4.096054238114927,
  3.8336614749941846,
  0.047316614895672265],
 [1.7306297576087935, 4.663502267648489, 1.693943096789606, 7.630590333874606],
 [1.9943187676958987,
  1.2930303285718145,
  2.976714601082817,
  2.826740171485768],
 [4.729840250985507, 4.81079941958277, 1.316728701488377, 28.872198732568062],
 [3.250865083626474,
  3.7194751634916545,
  2.6021199073535177,
  2.029324519682169],
 [4.0659593844004895,
  3.0820135252222385,
  4.759425307688151,
  5.720215875148372],
 [3.0656099033111652,
  1.1339160432925235,
  3.8428043093782676,
  11.247838673759638],
 [2.250513890708836, 2.118415537433153, 4.610814152030727, 6.990153368412251],
 [1.0671728566502776,
  4.720034767137713,
  1.4288545068263696,
  5.779737494544592],
 [1.9809945232368489,
  2.7369090643442577,
  1.8542298862042248,
  0.77171873393772],
 [1.4993304714913962,
  1.5556414644620071,
  1.3771896864763669,
  0.023873117219590487],
 [1.6869436362614696,
  1.8072278357010405,
  1.9041645398181264,
  0.007925872387123552],
 [1.510936114232539, 1.6438852768222878, 4.169655845185314, 4.819521286140075],
 [1.1676900800975725,
  3.493332713746299,
  3.0641543243375695,
  0.10754080581512486],
 [3.015811198334232, 1.7976512943558616, 2.3628508519398, 0.48170125776173295],
 [4.380897495615715,
  2.9159715964852024,
  3.826847017296483,
  1.8174022539796575],
 [1.4367497698684888,
  2.2853081366660875,
  4.154977886554823,
  2.5111979232137736],
 [3.5058953815234872,
  2.875644915826781,
  1.4990113081084688,
  3.3220463853055793],
 [3.41909174901186, 4.419027708236797, 2.5550667180188626, 5.93956168875216],
 [2.338659631505392, 2.2155115432876165, 4.330269904240433, 5.229480222555987],
 [2.706146859076891, 4.12829320679826, 1.1131515012503552, 12.300898123406933],
 [4.2618221761801385,
  1.1952253134386708,
  3.1350409432755413,
  8.018372683041763],
 [1.1008700497485733,
  1.1698179600787886,
  1.2530187851186185,
  0.003810318914328793],
 [3.5094055287119073,
  3.2217784726048926,
  1.4954747659995555,
  5.229232676234655],
 [4.833688401784746, 3.713788246845384, 2.10801652052613, 6.231839628699923],
 [2.1209872368430807,
  2.4046792530891428,
  4.8774005778443374,
  6.484229951058939],
 [2.51061094972506, 3.4159968081063483, 3.031378076300326, 0.1856993082900315],
 [4.642822612976003,
  2.6951634888649223,
  2.0285894938340348,
  1.0314515397313486],
 [1.4105834313255596,
  2.6269648211696435,
  4.791408027564211,
  3.3041607812997165],
 [1.988772317541077,
  3.8130916982143206,
  3.075929877546731,
  0.5403569461422069],
 [4.542355726810597, 4.393494437671006, 2.443877409973079, 8.632761945669683],
 [1.3320195918308473, 2.0090043928337127, 3.256611835755563, 1.03666045244895],
 [1.3375628135624162,
  4.447207398490029,
  1.7744746244700074,
  4.777440301239936],
 [1.3438079376649306,
  3.054320260618096,
  4.479814041536875,
  1.3653307146071028],
 [1.9110431701938886,
  3.954223206089878,
  2.5667681608795183,
  1.8394091526109368],
 [1.611654727689673, 4.349978077033101, 2.002264976183214, 4.4415244554423445],
 [2.659251670324113,
  3.2825564081853145,
  4.049239477653172,
  0.7815579603938372],
 [1.4191763750128623,
  1.5008367420711948,
  1.1205049365417166,
  0.10264355081401882],
 [1.4031491725677827, 2.9815725810979137, 4.7536762231792835, 2.203190676907],
 [4.631263585436156,
  3.5061692454907445,
  3.0810738897038754,
  0.4184487011730481],
 [2.656876823868639,
  3.4761837272090768,
  1.9140207025606673,
  3.241859083105976],
 [1.2635078831063233,
  1.7665206392440735,
  4.768714108335196,
  5.694102910004348],
 [3.5706301980272155, 4.157172401485283, 3.504465074759233, 0.760592175677512],
 [4.65006843594348,
  1.9602105294253103,
  1.7548926557332116,
  0.09801281549684565],
 [4.099488535286468,
  4.979498222913163,
  4.2568393300002345,
  1.0704499921763548],
 [1.3713150104862746, 4.04730907916643, 2.307388206768086, 2.075707861737009],
 [4.145749295930296,
  3.5129070956441932,
  3.8486901683642545,
  0.2337171802251847],
 [3.02904361802108, 3.5768404886607446, 1.473612204591113, 6.699592049649564],
 [2.4219437239697807,
  1.9485626493474464,
  2.65893087599622,
  0.6110842750202119],
 [3.4474335612527374,
  2.575906974425962,
  3.5051547333242494,
  1.488431848822846],
 [4.4799458212494425,
  2.8856547338612204,
  4.972197527478089,
  9.752082320339602],
 [4.175410158016309,
  4.1425503965239265,
  3.2583669983366415,
  1.632126664627408],
 [2.2975058541583278,
  3.7794422902955964,
  3.0285679480994636,
  0.6476810044097064],
 [1.750650845723075,
  1.8398070464216643,
  3.9048848248635957,
  3.7328657329754855],
 [2.984799703770027,
  1.3427992126180097,
  2.5844433671310547,
  2.3008033117397617],
 [3.340807156560921,
  1.0289949403052816,
  3.0239356372451427,
  6.647852757945256],
 [2.5023139782658657,
  1.3070066638340507,
  4.708350577609652,
  14.474810894465964],
 [3.5665812454849397,
  2.2578828589827586,
  1.956169998769008,
  0.16233410455989702],
 [3.086189206219763,
  4.917250547184677,
  1.0990117873163459,
  22.496694785564305],
 [3.4191229180874325,
  3.931954261165763,
  4.707682635617484,
  1.0287363196907906],
 [2.081755514982963, 2.517368482960843, 1.6835409865675897, 0.72368930244452],
 [2.5715317544252874,
  4.028659347482435,
  3.790499946967758,
  0.07292851204771518],
 [2.7554561224324146, 4.677966034850722, 3.7068140411238946, 1.2993852012894],
 [3.6473661743811006,
  3.112712139516138,
  1.3666689522530802,
  5.559802103136433],
 [1.445560871059115, 4.151385823090457, 3.1370867441386228, 0.743598406886922],
 [4.178560068670929,
  4.927612342478927,
  4.544936564351721,
  0.30595573762267125],
 [3.206143168628902, 2.462494035827553, 3.9238664892410564, 3.423534820634313],
 [4.69424022751636, 1.2772876388117167, 3.848695222694872, 15.519479659494552],
 [3.97049688038248, 4.571300825506351, 2.382440009526052, 9.511546973091853],
 [4.437164595232829,
  2.2868268560094918,
  3.0666635478675164,
  1.349220321455981],
 [2.9304493596097947,
  4.074186857776477,
  1.2349174778910839,
  11.81183639107835],
 [2.766428437027168,
  3.9958411429299256,
  4.562077939038588,
  0.44349182673692417],
 [3.256825315797216, 2.655529512319215, 4.114276812988544, 3.4651704355482886],
 [3.971201685401743,
  2.5608471910984774,
  1.6269368115986214,
  1.7318184130733199],
 [4.160324460200814, 3.588176504300669, 4.858914265599645, 3.358992837644011],
 [2.1388231466796133,
  1.1354107999826555,
  4.660452265358682,
  13.288419804867916],
 [3.937038311369284,
  3.5068296488705797,
  3.969733227701817,
  0.42181373998055477],
 [4.4189744341645385,
  1.7495814847559843,
  3.6461358011316607,
  7.947344949420818],
 [4.753071852279155,
  2.509200369865268,
  2.0896454731682903,
  0.41833285285107197],
 [2.5918709894787675,
  3.2331955080774004,
  2.57665284870504,
  0.5586107447135672],
 [4.143411829358316,
  2.2073506860804746,
  4.3403846916510975,
  9.425918151391663],
 [4.5863672158262725, 1.537470438493032, 4.823776543080985, 24.76594224550317],
 [4.824513530026998, 1.1505109048697904, 3.048405821174573, 8.688961202256866],
 [1.2033753289168994,
  4.19399296500853,
  4.7735194954201905,
  0.2020774034656948],
 [3.857273630345641,
  4.183403175359793,
  3.4978223008419693,
  0.9065000658435497],
 [2.274703252090393, 1.11919095283529, 2.2832709224852397, 1.5412049160030112],
 [4.366231647362248,
  1.9754323348811735,
  1.8168532796533556,
  0.05489950513521318],
 [3.5047697017206696,
  3.119922582448998,
  1.3591397096052935,
  5.433017456724261],
 [1.9753237628119367,
  2.3480206148352947,
  2.675841328957212,
  0.1061404871650735],
 [3.975673896052888, 4.11737587751562, 2.2676926455929434, 6.801042325918253],
 [1.3200137641851133,
  2.507830162632317,
  1.4124418522037945,
  0.7919261210660752],
 [4.160511945667689,
  2.5045215355509987,
  2.197461409340829,
  0.19613885053948488],
 [1.2787589885281236,
  2.8773945625268196,
  3.1696338240316884,
  0.05460542947848102],
 [1.1781778610578462,
  2.997290925662593,
  3.6208925597913546,
  0.22908431308885563],
 [2.7206125230128695,
  1.2794281417834132,
  3.3950882892404777,
  6.088755120944236],
 [2.3719585939853256,
  2.812294784687811,
  3.7412083583039144,
  1.0233583224965965],
 [1.5344109795847092,
  1.0682511498058882,
  2.3257932924718663,
  1.2132681525985243],
 [3.98363509501412, 2.131587472116445, 2.940929197464229, 1.304708271959304],
 [2.0843065240376806,
  2.1162755278405982,
  4.999363544119646,
  8.662582707033613],
 [2.7810114425203083,
  1.3933008991634424,
  1.5894823789773969,
  0.053516634282210776],
 [2.033471950969361,
  3.9403494003053225,
  2.2378556676455457,
  2.946993882138042],
 [3.4719665478164465, 4.160726145166473, 3.126624221302648, 1.856402859270814],
 [2.43585359360139, 4.013317876339594, 1.9283346439775109, 5.294516660843731],
 [1.1471897983602064,
  1.4955123854973267,
  3.3747806845414456,
  2.0257360469975714],
 [2.471598208502718,
  2.4691127740420398,
  1.2686010011288666,
  1.781068910210403],
 [3.4264907318377245,
  3.6696126205516983,
  3.6930949568768563,
  0.0009447179640443052],
 [1.3340941476959944,
  3.388657526507362,
  4.702406559788093,
  1.1512809069450294],
 [2.209179573174939,
  3.266591098213129,
  3.3325128114772946,
  0.004800185215993951],
 [2.53140480098846, 1.3167481355597093, 3.3313811280565737, 5.137164674750643],
 [4.34983992197108,
  2.226055023120241,
  2.1542287656490577,
  0.011220436573361803],
 [1.8527250660486634,
  4.042785376607833,
  4.161984746276314,
  0.013162212536143034],
 [2.719680708744145, 4.058463389349015, 1.8680515610643225, 6.524383444992483],
 [2.5708665992875974,
  2.591362390754264,
  2.1633088485181218,
  0.23552973142411543],
 [4.728604902604268, 4.926055813106712, 3.549038763489616, 4.483133458355754],
 [4.970957770589628, 4.73030803095788, 3.5027862885000007, 3.745143515066999],
 [3.226344420113034,
  2.4632452025137606,
  2.462389969143088,
  1.1799130614260755e-06],
 [3.6245092814133604,
  4.5392191379855955,
  1.9135025281671392,
  12.494386131478562],
 [4.31832803625689, 1.916474316574722, 1.0486494282508532, 1.6261096847778995],
 [1.844754339057273, 4.509722724167298, 1.2285412125638326, 9.93045291186285],
 [2.141691713186919,
  2.6331930207313046,
  2.8396624151385823,
  0.04564974212218613],
 [2.752264533417684,
  2.5277013269807607,
  2.1780213817955425,
  0.16826803720564235],
 [3.4554431263769456,
  1.1248191031842936,
  1.4633198655321862,
  0.19796711577820886],
 [4.529797634343122, 1.3581331987005258, 3.387304590079159, 9.325803629117342],
 [4.675862874230897,
  3.5738403385280075,
  3.4395895339790212,
  0.04213718945658864],
 [2.052950266098656,
  1.4061658338434162,
  1.117417868279309,
  0.08558276209762108],
 [3.8624973753339917,
  1.1159572375657145,
  3.522027786312733,
  11.180337559138092],
 [4.989687993550946,
  3.5580027788401027,
  3.396878009003099,
  0.06476912265071372],
 [1.66152848454993, 4.818988399298214, 4.228530533611428, 0.28963805345776006],
 [1.7605594835784322,
  3.1935136220255145,
  1.5819247311064846,
  2.2862790535546367],
 [3.964950986859405, 2.581446402800967, 4.836257830477114, 10.079251498024648],
 [3.380884662494513,
  1.1551540280524755,
  1.266890065938275,
  0.021105074734523034],
 [3.0969368487547606,
  2.006151195476489,
  3.0736882149091413,
  1.7646892585561227],
 [2.608473041753436,
  2.6897201506948236,
  4.119947837688928,
  2.6678826281562977],
 [3.8012692022468793,
  2.6672727381379806,
  4.267141223915642,
  4.864824738208511],
 [4.36013494426472, 1.0421654152281015, 2.312133839097079, 3.516055979211735],
 [3.247093395768088,
  2.066977218583809,
  2.218951604272548,
  0.037497781819848815],
 [1.8506789439195357,
  1.0482918038916202,
  2.1678025406605697,
  1.159731929673808],
 [3.80223811571091,
  1.5950321894157633,
  2.8534874514870285,
  3.0108205913270547],
 [4.87357255506185, 3.989173357647289, 4.187769082081076, 0.09610748864723304],
 [4.413300864061962, 4.009300989676229, 2.224544396766779, 7.028967405264722],
 [2.8761288119302826, 4.654969130663606, 2.45791715536136, 6.941590645575492],
 [2.1884810048073615,
  2.3071120891023145,
  3.4831065528973557,
  1.5132941048116069],
 [4.026331909282866,
  2.4997274785602404,
  1.8859493693510685,
  0.7584070600887872],
 [1.7730398004941783,
  3.6365332889186632,
  4.206614137457796,
  0.28811202956129045],
 [3.553232714120408,
  3.429044393948344,
  4.1399482711583175,
  0.8978740541901105],
 [1.6288813992315685,
  3.135237030113618,
  2.643922970216561,
  0.19659743770066557],
 [3.7770567101046764,
  1.2191479896106467,
  1.6528935804247613,
  0.3552987306989766],
 [4.39964427952155, 2.302874485979127, 1.2465227723605872, 2.4547352038068917],
 [4.529206635930317, 2.912881165560942, 1.0542748189438176, 7.822885449183958],
 [1.1499257356488206,
  4.397302142376422,
  2.689898977245708,
  1.6761464531038825],
 [2.377369508232605,
  1.4090206604944977,
  3.1487189255217234,
  3.5976139059736414],
 [3.5245174101470456,
  1.5216116401299744,
  4.601890503462688,
  16.720518321558384],
 [3.407489039110142, 4.913029571201923, 3.393005869333713, 3.936454099998483],
 [1.3470771221721676,
  1.442937214282746,
  1.5500113880453816,
  0.0077220378938355596],
 [1.8569058025507634,
  2.0326871836665212,
  3.9994137968032093,
  3.5912684220004785],
 [4.801988417799713, 1.9759825776560476, 3.754922943398356, 7.598255481824133],
 [2.7513977603325954, 4.206070521052183, 2.35638817762068, 4.70671265697844],
 [3.701912223840339,
  1.3011203316727493,
  2.6053076455057984,
  3.1482996718065763],
 [2.649401454706276,
  1.4011517202013697,
  2.002803203662718,
  0.4795211404436963],
 [4.096034007434815,
  1.024413966540421,
  2.1344417101535083,
  2.5234878909053835],
 [4.20685874501538, 3.330404641567676, 1.1068110785199106, 10.400129581448894],
 [3.1695823705771424,
  4.677874817652187,
  1.4265063871423926,
  16.753456260256183],
 [4.366801626564732, 4.307622951424074, 2.2066774656565515, 9.637469910865617],
 [3.538100066164477, 3.764084969815023, 2.5625404415802193, 2.553993902366334],
 [4.149906738502027, 2.55813623375748, 2.0248538508929657, 0.590096195914471],
 [3.1508466969427418,
  1.6805229213661708,
  4.654082951279845,
  13.929986593380727],
 [1.2028166616773555,
  1.5814643152476653,
  3.913509345442825,
  3.270719528063271],
 [1.0457574121635727,
  3.8652916090699865,
  3.01333136941293,
  0.3795243192045155],
 [2.7112501936189273,
  2.9458453326582874,
  2.80196362702576,
  0.028064076487211796],
 [1.7450039308960097,
  2.4300202126986754,
  4.236745364782832,
  2.8480695795638966],
 [4.962089582771021,
  3.4284804509373354,
  1.5523464097766073,
  8.732977311392142],
 [2.083754480502162, 1.6143770998402291, 4.913272435184586, 11.33844831296576],
 [2.6956783159010986, 2.4143273900610787, 4.431648633733275, 5.48514601983885],
 [2.366431559741431,
  3.0851844281662557,
  2.6399888804049176,
  0.23451227397907112],
 [4.424448383419573,
  2.0375749609260354,
  4.833301314763871,
  17.290934191991546],
 [1.0563860635869005,
  4.505866305165229,
  4.378521125233506,
  0.008565598038767306],
 [2.3847850752301394,
  3.894273864593141,
  3.8382737537055713,
  0.0037393578067847293],
 [3.4123287850273583,
  4.602727596626402,
  2.606767227941561,
  6.7971163118698525],
 [3.292085706181912,
  4.1279400173695695,
  4.445944274101701,
  0.16645889380738366],
 [1.9377446150011277,
  1.2229246281390558,
  3.522649110745054,
  5.124106650827818],
 [1.7968759007154476,
  3.9438382657218947,
  4.3055402547927475,
  0.11754113566367579],
 [4.086993107024227,
  1.3437460479600976,
  2.2039975213396272,
  1.5122540623790675],
 [1.5414531609109585,
  1.5259428735430838,
  1.2365610955368131,
  0.06454204151534768],
 [3.706117921884141, 1.1621144225040965, 3.332333311840403, 8.727629798396944],
 [1.0259094949333565,
  4.74488233087726,
  4.445697724393497,
  0.04591531233334857],
 [3.765625936284538,
  3.4062455991783027,
  4.500611603264325,
  2.2549263822793937],
 [2.453329789809553, 4.238961578227521, 2.858294819355585, 2.3383185467682037],
 [4.284596829048262,
  2.2617738244356334,
  4.743287491236309,
  13.192080997992424],
 [2.7538272326391042,
  1.8956486450934649,
  3.771450537059445,
  4.844853327646594],
 [1.0571321971544636,
  1.2228026566913108,
  1.524525237198231,
  0.048118815872284955],
 [2.5886827433525057,
  3.134157724846414,
  2.895720425084263,
  0.07358634339834323],
 [3.7569206365939407,
  4.30751528136512,
  3.6884100745846977,
  0.7199974167423129],
 [2.214565980414637, 2.814768660199436, 3.764092288966474, 0.9979004299332462],
 [3.3478760143990667, 4.815456562292317, 3.271813916789429, 3.988714082416396],
 [3.925019452888457,
  1.9862769713748674,
  4.6809044573894365,
  14.249817052335649],
 [1.9459290908709512,
  1.4910655973863713,
  2.233787408715602,
  0.5367219674173904],
 [2.7214352444692045, 3.090039777570782, 1.420645761942033, 3.792151800462186],
 [4.796782835816908, 2.711730878954089, 3.036615303544868, 0.2531499487582766],
 [1.3900555567450072, 4.296121131064985, 1.8732957785380377, 4.07987053009245],
 [4.467136352886143,
  2.8231952020332733,
  2.201272763543521,
  0.8639162945939417],
 [3.7553348560611632,
  4.1492296605876655,
  1.4264992073260108,
  13.919638942609522],
 [1.092807513579991, 1.1295748975083724, 4.270942356423555, 5.392015822034859],
 [4.548959606884461,
  4.018499196979821,
  3.5876297645253703,
  0.4222536906048198],
 [3.83573462631414, 4.16576517027937, 3.1935917561870046, 1.8126169549352427],
 [3.4690732449535293,
  4.716555363607329,
  2.4936468638170224,
  8.570904316651562],
 [3.9162398421484363,
  3.598968110356485,
  4.605869113910277,
  1.9852391593518866],
 [2.3886868384425677,
  4.184469669833215,
  2.8955363307435724,
  1.9842214274567012],
 [2.9901955952647428,
  2.2083987149109405,
  2.5752411221442384,
  0.20120032181353842],
 [1.0462146577884783,
  4.061678157094828,
  2.0520720640890415,
  2.1125776569773578],
 [4.044391744079993, 2.3501761560940113, 1.168132916796106, 2.825465093520271],
 [2.715424153130345,
  4.037151640878411,
  4.129564123483881,
  0.011594952020886487],
 [4.614632274026618,
  3.3021186878465705,
  2.305803206999274,
  2.2903447594057074],
 [3.549422130112032, 4.595609684797352, 2.465317024122089, 8.053899376527966],
 [3.391026244104423,
  1.7361004936337538,
  2.7159931746443338,
  1.628014178867366],
 [2.605067521833329, 4.3264510352487555, 1.990166139196035, 7.109525092720886],
 [1.2078113373475863,
  2.1014119473925597,
  2.5159977157215216,
  0.10380012722118002],
 [3.987429993722181,
  4.099053686024276,
  3.0955272507235096,
  2.0078012040834463],
 [2.845388982609527,
  4.301518124587835,
  2.3765000170384623,
  5.2720714566240545],
 [1.2211869167096112,
  2.719973044560928,
  4.4002217917438955,
  1.723849342874258],
 [3.0194155902726236, 1.5852751846997046, 4.68792408573016, 14.5330967169036],
 [3.0714106980997435,
  2.1192456909648003,
  1.9723524389751237,
  0.03313687794099912],
 [2.432539105241458, 4.712458126586808, 3.892936181735311, 0.816866357113019],
 [3.090781436239714,
  1.0674584095819784,
  1.743197328440751,
  0.7056610794951783],
 [4.114592040973582,
  2.704238932177536,
  2.77463284578371,
  0.010194525291964004],
 [2.073958737791348,
  2.1368378976096007,
  1.2862715042025612,
  0.7502164017638763],
 [1.530009540186021,
  3.8417681451378956,
  2.3097283814924623,
  1.7955777617078812],
 [2.9558049386071024,
  3.423148847603093,
  3.333543223704797,
  0.011866326968621034],
 [4.607779927992471, 4.939226656170124, 2.166779636866637, 17.708763754428578],
 [3.1413670854978832,
  2.2531918262575843,
  4.302128018957123,
  6.593948656983046],
 [1.8167803262442277,
  3.3546087911571907,
  2.8078458428128488,
  0.27156298644420196],
 [4.298272835014934,
  3.483819732540959,
  3.2381501147911558,
  0.12970803605475373],
 [2.70077799025108, 1.7553244773767318, 3.021549617705069, 2.1651139291392334],
 [4.922055202766926, 1.634485851193034, 2.600943639085908, 2.298699834685907],
 [4.991618580945701, 1.318634646318515, 2.2109039979418514, 1.987025078873021],
 [4.075983479238106, 2.619149540851273, 1.1415912827510715, 4.449299557705937],
 [4.0797084182244685,
  2.194405217181214,
  3.4600855604606204,
  3.267737782746188],
 [2.4922563632501404,
  1.979811836486281,
  2.207672321922627,
  0.06469947466716772],
 [2.0670065054558178,
  4.538778535757187,
  2.2103865479168365,
  5.603044093251119],
 [3.751867140663585,
  1.3328510603756092,
  3.8221014396831454,
  11.623973715222116],
 [3.2852575383876, 1.373346287067517, 2.761892479008769, 3.1670876906510315],
 [1.2189821005343484, 4.312874768552716, 3.319931516653313, 0.600919351893234],
 [1.0737971860238318,
  4.714415050897608,
  2.2987897948825786,
  3.1329350330577874],
 [1.8095267915411566,
  2.195966776866261,
  1.8276155904776865,
  0.12276064676885855],
 [3.7022983414667694,
  1.7058011464809897,
  4.983915346045153,
  19.892509531245782],
 [4.965735684220685, 4.166568791221827, 1.018665572565682, 24.603469084133753],
 [3.0924925922682482,
  3.4788057208913963,
  3.3977185861596566,
  0.010166760233258595],
 [4.07234995368529, 4.785422992825879, 4.198633094233912, 0.7011006244861604],
 [4.425376017877973,
  1.9332691680654048,
  1.2678300349841876,
  0.9797986952322438],
 [2.8338266383293105,
  1.508521941361376,
  2.2948778883831618,
  0.8761563924787371],
 [2.732037810694858,
  4.166737421494121,
  3.9701212970134327,
  0.05280742279536828],
 [2.7605891521762596,
  4.365923355004872,
  1.345837277779529,
  12.589556286021866],
 [1.4441374742250845,
  1.2648180673457676,
  3.2407172533955273,
  2.819084584352603],
 [1.6162664817936925,
  1.2379245192860937,
  1.3193884235889,
  0.005363069340624192],
 [3.3668831621668565,
  1.6489735358399873,
  2.7824708731038847,
  2.1629130379980546],
 [3.91044477019682, 4.976052329979388, 1.2446862089770399, 27.222743355560493],
 [3.4150974416840265, 4.003794199444313, 2.200823408655211, 5.5507348977091],
 [4.732179785983599,
  1.6566898328908901,
  2.153154137664315,
  0.5831862793303952],
 [1.987568719666133, 4.619701135739666, 2.200451264522698, 5.8163912274602625],
 [4.658942764722674,
  4.637799323618682,
  3.9685352985471702,
  1.0434036257257415],
 [1.128934450785955,
  4.3807960148500324,
  4.513897562027486,
  0.010000113704897996],
 [4.560636196672796, 3.673941339192303, 4.337839917211305, 1.0050760193653325],
 [3.217040529326677, 2.10658620833072, 1.4799750761053936, 0.6315718272377062],
 [2.43562804544634,
  1.7172968904379164,
  1.5403485193664412,
  0.038130641214703775],
 [1.1965024868110228, 1.3150920069868208, 4.436826095118591, 5.83009220596246],
 [2.9726387877783336, 4.399847815498894, 3.49280075829766, 1.2228460411987045],
 [1.4224215375193046,
  3.2548746762100627,
  1.958107218419641,
  1.1959761819214227],
 [1.3163733428071898,
  2.8499407568335604,
  4.3872248841250165,
  1.5554547069109173],
 [3.2654642855435028,
  1.9761300875671406,
  3.591956564005681,
  4.262892517648291],
 [4.7828180840833925,
  2.6508703685670434,
  1.4847163654794575,
  3.252113407444165],
 [3.7312663331153004,
  2.101143216590189,
  1.4746542125327515,
  0.7322395112369641],
 [4.142986360552769, 4.581585479105442, 1.7977020150968013, 16.0540869400492],
 [2.601360282254775, 2.598595173381871, 2.23130137717301, 0.17546790681058264],
 [3.974451398350184,
  4.670439964717373,
  1.9144703158900747,
  15.093711885797473],
 [3.227775507904405,
  1.8112529411440956,
  3.047061006763319,
  2.4647641975440258],
 [3.5757770635106714,
  1.1995173316698229,
  1.787214653689269,
  0.6175154986382345],
 [3.3402426824045826,
  3.828782811609587,
  4.547797211756446,
  0.8634221829049398],
 [4.452753465212513, 4.241062970225709, 1.9845047376555414, 11.33683290001074],
 [4.168065763833881,
  2.961529520772927,
  2.2552662207789207,
  1.0395319589042529],
 [2.1286569715080788,
  2.526427810865528,
  2.250214286367682,
  0.08120178288983691],
 [4.479270781428326,
  2.4213966401620124,
  4.641295835333134,
  11.036816681037326],
 [2.298439553404267,
  3.9915774247837903,
  4.391718928615591,
  0.18400528248514056],
 [3.3783878608319773,
  1.6071518584651723,
  2.622134903713098,
  1.740191678523874],
 [1.906071328084824,
  4.429906716090027,
  3.1657382548525996,
  1.5230671646503608],
 [4.157637012921025,
  4.7872079472913756,
  1.6100722552149702,
  20.983991486504824],
 [3.683372890249315, 2.8126343492847132, 3.45398595845646, 0.7575443599751785],
 [4.214992553632614, 4.85005720757908, 2.85757571757796, 8.366723312479792],
 [3.811528010303846,
  1.6623116317894153,
  4.975703968697981,
  20.922561205739136],
 [4.859077153685451,
  1.5547121772216848,
  2.193337461250855,
  0.9908684879050177],
 [1.7333887675646187, 3.12490898498838, 3.990194351305588, 0.6489103487989417],
 [4.129519176157549,
  2.1255302086886267,
  3.442636321252299,
  3.5818799177383034],
 [2.2167829508859387,
  4.948401963416375,
  1.5972181190518988,
  12.447716378667803],
 [1.5385330644096746,
  1.7687887327384253,
  4.092813309229688,
  4.154877953000213],
 [3.782643347072615, 3.943817910166454, 3.304328746900726, 0.7734491705971729],
 [4.084317445486907,
  2.9213746401597978,
  3.9826103262575514,
  2.2999224094311845],
 [4.294629173973943,
  3.4324920343395346,
  1.4681335180192665,
  8.285852203418726],
 [1.0093585954633264,
  2.079367720774587,
  2.3321954789034915,
  0.032260047126247554],
 [1.3499201662224931,
  2.1432330175818137,
  3.2520315744689787,
  0.8298190366448873],
 [4.176888949429866,
  1.9378212882535273,
  4.316742778903652,
  11.819065855008027],
 [3.789186638811972,
  3.5296755251028067,
  4.0465429441398495,
  0.5061442596883949],
 [2.2734122027394803,
  4.796105656624226,
  3.9476023266637648,
  0.8183805387437529],
 [4.185232205239656, 3.0012943228630635, 4.938605724200844, 7.853955415593143],
 [3.363889965865338, 4.930865496076177, 3.342496510717634, 4.243405965156203],
 [3.185518899232731, 1.9981192671053254, 4.956747330254153, 13.94218801259449],
 [1.6292585173195047,
  1.2338515371491021,
  2.4726216358657593,
  1.2500904347147495],
 [3.8437004714129124,
  4.457473474603001,
  1.4338277223890765,
  17.570388236127272],
 [4.589265842789512, 3.22995287743188, 4.219203421145743, 2.2455659555259775],
 [3.685341146738745, 1.7309329100856061, 3.9868562268449166, 9.37770067573624],
 [1.9280741756723057, 1.1802479579695384, 4.993039826416442, 14.0145752461749],
 [1.206027842132849, 1.9513916787126, 4.405721459994963, 3.6323958660101257],
 [2.049153478034736,
  4.95440128300058,
  4.877757849207368,
  0.006018585015792316],
 [3.5549681219357723,
  2.2561938520924727,
  2.7097273135842697,
  0.36561531919036183],
 [2.9034386499712523,
  3.0235942136327667,
  1.583971698623385,
  3.008707152656196],
 [4.386682128955598,
  4.932099389462669,
  2.2300187541718617,
  16.014108986358465],
 [4.111494608052302, 2.416368954356841, 2.945683759606586, 0.5759672803613383],
 [3.500419952005139,
  4.718063525449024,
  3.1773497168514817,
  4.1546467608388244],
 [4.055537906109212, 3.424821548135227, 4.590445692580823, 2.755088403538257],
 [3.487335780379633,
  1.3103327239580889,
  2.6229399308663752,
  3.004231108764748],
 [3.359626307831972, 3.2652446458334174, 4.814445761869227, 4.031592049412449],
 [3.5936218788088845,
  3.4298066554503372,
  2.6336889250788436,
  1.1388249555235608],
 [2.0404953972087654, 4.589165403227209, 2.092648129695126, 6.358794272934344],
 [4.225929629107169,
  4.7077651618326435,
  3.7907699912632262,
  1.776750155093844],
 [2.8937651126313257, 2.361255966336854, 4.10680554483112, 4.408569155679489],
 [2.509488517187492, 4.044575675911001, 1.9784811708853902, 5.356185166905968],
 [2.3008604296313093,
  4.671135913861941,
  4.285931975069021,
  0.17070322179762798],
 [3.3869179461370407,
  1.5061016496853776,
  1.6946366273881464,
  0.06019474062349721],
 [3.824351926523316, 1.2527221746386195, 4.325889160093527, 18.05926923234334],
 [3.0344447687202294,
  4.146049092897458,
  3.701435411405949,
  0.2999265324273682],
 [2.5884771793789807, 1.999684457147524, 4.523730655339527, 8.245347127917471],
 [4.120650720383837,
  3.1646623524513013,
  1.6084169960455021,
  4.9899011849510035],
 [4.009499856819243, 4.25127792421064, 2.0866718974087566, 9.393294383539496],
 [2.356307377548929, 1.824190752418113, 3.569435229583013, 3.588512737118663],
 [4.63983725804947,
  4.494137197436276,
  4.6970813337979065,
  0.09554891679065022],
 [2.229403251051148,
  1.7632256171496627,
  1.697423659041847,
  0.0048265439943195355],
 [2.983773657705826,
  4.7889692000004445,
  3.8761006940760456,
  1.2432324236015526],
 [2.7350945149510992,
  1.5312526090495444,
  1.8845061791834916,
  0.17065360315079595],
 [2.1123564857048684, 2.3113775761764743, 4.60897430118843, 5.575512885822209],
 [1.8683023013383901,
  3.5728246480352754,
  2.4450624652076387,
  1.1880978439160284],
 [1.9782571152101358,
  2.0173155316838405,
  2.746263482623124,
  0.5255884099382397],
 [4.10092414396822, 1.2445818477199273, 3.00635135525899, 6.36428937899069],
 [3.078438772191744, 4.750451431167129, 4.308187568485345, 0.3010671933297781],
 [1.763070172333757,
  4.515128462931642,
  4.3148585442855385,
  0.035356644774642364],
 [4.298577085201575,
  1.8878535633784286,
  4.127552135007528,
  10.781367989318978],
 [1.7617973795950346,
  1.5245206556422213,
  3.5324435814947592,
  3.551567235638334],
 [3.0242071020642602,
  1.7185304059686106,
  3.5492580234397164,
  5.067911235240604],
 [4.665005299748611, 3.9121974304688427, 2.399721834326093, 5.335792077278981],
 [2.1279892930250455,
  3.0730890630403205,
  1.8068048618308863,
  1.7060895374525384],
 [2.1443258913765337,
  1.9503328896128953,
  3.8024703669801894,
  3.677961959039042],
 [4.840580232304804, 2.778320738230259, 2.245031298354595, 0.6883247649178228],
 [2.1770201723366522,
  4.597576630917673,
  3.061103028495778,
  2.5697014169607275],
 [4.488027781615273,
  3.0533615617932477,
  4.077583074433171,
  2.3540372342755673],
 [3.5193884228122463,
  4.449910183911348,
  1.2634668019387112,
  17.866916910209248],
 [2.383518584400249, 4.743750174207948, 2.744648577110149, 4.762755410675172],
 [2.5502749693334787,
  1.4790659762750509,
  1.2472846052451674,
  0.06850370607882929],
 [2.8394173950014427,
  4.016443943287385,
  4.353167787274643,
  0.16097075615969986],
 [2.9085600914208953,
  4.895034029443476,
  2.3002687069468624,
  9.791386986361907],
 [1.8635194761078089,
  3.0268347598881533,
  3.403503131812269,
  0.1321971980245779],
 [1.524568642090328, 4.83999548087677, 1.8579250840939574, 6.778799208968917],
 [4.468959067863462,
  1.3438154600980194,
  4.985827202362892,
  29.638694109525474],
 [4.540469943078968, 4.199083928695973, 3.776733887993955, 0.4049635082389551],
 [4.7306585451092165, 3.672444595360623, 1.840656297530494, 7.936740247535414],
 [4.804169612588753,
  1.0703053607379243,
  2.314405934763485,
  3.7179138063896895],
 [1.9899522434198413,
  2.9514326297356344,
  1.886265128733315,
  1.1288818042921052],
 [4.056351484310727, 2.108476729250972, 4.361739719101839, 10.29744191473801],
 [2.0970141962092717,
  2.564062941195263,
  2.78591596307293,
  0.05160622269702295],
 [1.0421692291207392,
  1.8290532381479307,
  1.4919732941536772,
  0.059207139133861574],
 [3.5389028803685143,
  1.7534356915057798,
  3.8777532434250355,
  7.985047859036052],
 [3.279805504891467,
  2.8608938942699718,
  4.7017807846842326,
  5.5574082922219254],
 [1.8492121376849577,
  1.9825686099003308,
  3.8123501035298077,
  3.0956738698143904],
 [2.1570567574129296,
  4.470916573164269,
  1.3383669206689608,
  10.583455886869736],
 [4.236416411368941, 2.6246525232270304, 4.05988180687029, 4.363261278026356],
 [4.768584101363077,
  1.5548835496836793,
  1.5764189669511506,
  0.0011057731309419793],
 [3.3496105268716856,
  2.736381012910676,
  1.2181606024785232,
  3.860414768084369],
 [1.8164094519911935,
  1.507881330206002,
  1.7458355628998885,
  0.05142456494581712],
 [4.40762847046968,
  4.4396096922912065,
  4.600462721764955,
  0.05702082196704117],
 [2.2310104462998233,
  3.0957927324779724,
  3.1681062534428577,
  0.0058332504612772515],
 [3.4745205210653896,
  4.5246615986967615,
  2.1353683916317117,
  9.91753591997769],
 [2.8804449678820077,
  3.300280635712869,
  4.554359759425971,
  2.2650587095961345],
 [2.0652447924945214,
  1.682951562025253,
  3.403118032934669,
  3.0555014671419993],
 [4.703208170271958, 4.721171437432565, 4.490671894376774, 0.124940817576375],
 [4.992653837890408,
  4.8949194873569315,
  3.5218359661995517,
  4.706470816125675],
 [2.095446424034629,
  4.858575193403794,
  3.4519178332799743,
  2.0731141291626494],
 [2.7718375870348138,
  4.560838693621106,
  4.62864400786278,
  0.006371845694497284],
 [3.102601108406325,
  4.964357393658736,
  1.1588437970558463,
  22.465831827378253],
 [4.332663769440491,
  2.284606700293604,
  2.139211797646435,
  0.045795557868403745],
 [3.987445366397134,
  3.1210661072394243,
  3.1845773003989777,
  0.008042022678239063],
 [3.9050077323917565,
  3.454892306338322,
  2.7949545191140066,
  0.8503503503676856],
 [2.6847928000434007,
  1.5441983517037907,
  1.4108071520459249,
  0.023885543929833785],
 [2.319072750139197, 3.850248952388884, 2.51120456138144, 2.0790949640725778],
 [4.266580983209365, 2.5632678456025526, 4.295682144950306, 6.40255793731034],
 [2.2556712808539436,
  1.1045394792573306,
  2.5780850691258386,
  2.4489108109574405],
 [2.178450892493259,
  1.1679196536013512,
  1.272584361965051,
  0.011932139277861992],
 [1.2973604386269284,
  1.98811546594436,
  3.4073718174531704,
  1.3066291651638604],
 [4.712602180996708, 3.136963679977881, 4.118582022847036, 2.2704718125585326],
 [1.4929971422141697,
  1.4060874120800704,
  3.2308595143506884,
  2.4856858847126118],
 [2.9069680159102584,
  1.5684735717596143,
  1.170228248903931,
  0.23052160025822063],
 [3.2675481693369295,
  4.149039693389387,
  4.866642871272029,
  0.8413190242863415],
 [1.0782728210784378,
  4.2543292666853025,
  3.5489087546093057,
  0.268284035637416],
 [2.4242774074609583,
  2.6326530408825835,
  1.7061478768195246,
  1.0405141895926324],
 [2.6257018997851325,
  4.802018748510081,
  3.4766530857473548,
  2.3061462853147314],
 [4.165753627735679,
  1.6491991943647246,
  4.024480041870481,
  11.751505803611229],
 [3.546103365860198,
  3.4632076136434664,
  3.3502119658417113,
  0.022638353004951418],
 [3.870543458027394, 4.400775862390898, 4.09238507845647, 0.18405377707041073],
 [1.1316162566922907, 3.10430816916343, 4.774746882595178, 1.578811478315188],
 [2.889732838166934, 1.9939669382640721, 4.4529579547048215, 8.73658248807475],
 [4.883042576576184,
  4.5253029604819845,
  2.4798303650920284,
  10.215222864432784],
 [4.555320066107562, 4.835968244765035, 4.445096973656344, 0.3479816983477374],
 [1.5326963090119112, 3.83240573060652, 1.0653432704107253, 5.867647840363455],
 [1.7589144882765897,
  4.150164422440033,
  4.3048960753437076,
  0.02105586368296277],
 [3.182292622723199,
  3.67595614859631,
  3.8714127011889428,
  0.060786982518238206],
 [1.420511816329319, 4.758612760573741, 3.813214882089578, 0.6348105004057967],
 [2.3807091577126505,
  1.9789432387307682,
  2.837903683293567,
  0.8782591368407825],
 [3.7975506448941827, 3.480882060778342, 1.273243726537252, 9.253998657547083],
 [2.4316613125917335,
  3.4080905687875873,
  2.8791166808042203,
  0.3402056783553317],
 [2.4235618045129246, 3.628894786550171, 2.407025289502044, 1.809146556916232],
 [4.818192837084311,
  1.1493045430220241,
  1.6205011073012052,
  0.5348825285148425],
 [3.936990306198027,
  3.262628325191112,
  2.879572327776218,
  0.28884102835566255],
 [3.9810423473684864,
  4.1021378958935415,
  2.2784489565885555,
  6.620157622201459],
 [3.142056795621573,
  3.4671416615532498,
  1.3648525904535957,
  6.943347488242309],
 [4.229031553531453, 2.4009281702387533, 4.451988018468875, 8.89544329684223],
 [4.895655877673953,
  2.2544235833160204,
  2.756591209502795,
  0.6172744620225813],
 [1.1191568342672142,
  4.393620005629321,
  3.144390356840685,
  0.8732639290693006],
 [1.5915752803025982,
  4.616877707980141,
  4.280567122286906,
  0.0900074098789144],
 [1.5397475295202674,
  4.382248262427016,
  4.545271915593718,
  0.02046071293114917],
 [2.8162945345171617,
  1.0354326962511693,
  2.100700255757882,
  1.5979584406038172],
 [4.506959367511961,
  1.4686412448617157,
  2.1512201449162784,
  1.049927631519519],
 [1.5237832261459108,
  1.1086126882837064,
  4.083915001176008,
  6.744587489045602],
 [3.419789869789791, 3.695836642536482, 1.6546051388197967, 7.124492781514912],
 [4.118258904498715,
  4.446902320654652,
  1.7191325232854733,
  15.321422309843097],
 [4.083163407820476, 2.124378555585466, 4.761125517264885, 14.193963154345793],
 [3.259312332544322,
  4.326486725841474,
  4.593653935306605,
  0.11632211576271734],
 [2.290451508058274, 4.006474109948517, 4.864184291040527, 0.8425045139214637],
 [1.7954174453851435,
  4.575900178703959,
  2.1335099681088145,
  5.355074859081831],
 [1.6443573135000622,
  1.1403335757877118,
  2.254389646695571,
  1.0204231383737055],
 [1.24883722385613, 1.9892785116560905, 3.4523445712582705, 1.336606936839499],
 [1.9569727997084092,
  4.616770865306031,
  1.5829171149777341,
  9.006251624551274],
 [4.593994807224399, 4.662008696353414, 1.386864025600262, 24.638909444849013],
 [2.399335346804524, 1.273188812692509, 3.116884213568303, 4.077925628650713],
 [3.217582610985889,
  4.501761414394961,
  4.430504924736578,
  0.008168617451649342],
 [2.388897246964891,
  1.0553264090414989,
  2.0294217043764147,
  1.1333664850212453],
 [3.892101191748874,
  3.8842682709509213,
  3.6079216964592797,
  0.1486148811647931],
 [3.9164124020117894,
  2.52925313567201,
  3.3975443107594505,
  1.4763495487852207],
 [4.24694848908347, 4.410449284031434, 1.619191435150403, 16.544243460803248],
 [3.1424933896139327, 4.637487693212654, 2.3337127804122804, 8.3392014743414],
 [3.4866796669733424,
  2.5385390824272376,
  3.6092432564745023,
  1.998577735199169],
 [2.6403850804298044,
  4.04393687068449,
  3.1805461285700853,
  0.9841290449767514],
 [3.73388157805163, 3.6740953817077764, 4.09222889146287, 0.32640777271877935],
 [3.2141896589811947,
  1.024616300745921,
  4.839945186186342,
  23.394052755767987],
 [2.532617379534056,
  4.751333872695472,
  3.1076241613463305,
  3.4212895371596357],
 [1.5589717957054527, 1.5125367581007123, 4.991872065984969, 9.43628025955547],
 [3.303619016100905, 3.787516019491475, 4.605851111172903, 1.1061711092157904],
 [3.500247835581775,
  1.7043311143436561,
  2.8548728019506355,
  2.3167198418143458],
 [1.5417985633827413, 4.25482467509523, 3.1861497394008573, 0.880417950150071],
 [1.5445344885547354,
  2.407010277995994,
  3.706057096602235,
  1.3032184564783775],
 [4.051175611906226,
  4.54804322194548,
  4.579897135423262,
  0.0020553068329230806],
 [4.50621852227928, 1.2943907756579187, 3.447370258199993, 10.443883889929866],
 [2.8342186009308694,
  2.2061120606717384,
  3.413600481321589,
  2.066185444437195],
 [4.09799520908328, 2.542895177990041, 4.292411406436807, 6.271586279807729],
 [2.7838488233019705,
  1.5029800412639052,
  4.675254133228522,
  14.007384827088798],
 [3.9956228911341123,
  4.573411658868162,
  4.599740954367595,
  0.0013849464274607465],
 [2.8604910832774566, 4.91427744363479, 2.601306116637461, 7.651579601688492],
 [3.054185428349062,
  4.411786071724315,
  3.7093292727923317,
  0.7535371109137773],
 [4.847312176809078,
  1.6392574906401438,
  1.7790278246183888,
  0.04734793036552079],
 [2.57850794251694,
  2.4162772616403645,
  1.6651495403637733,
  0.7273878771499085],
 [3.4839866887893227,
  1.036839292429152,
  3.0190228490462934,
  6.844381827781776],
 [2.1666756946448094,
  4.87178009244381,
  1.3243520556295199,
  13.632989621221553],
 [3.0162940489957486,
  1.5171967412673908,
  2.9680618953890687,
  3.1746641087238294],
 [4.5305321255258395,
  1.4296487741219344,
  2.467796883740918,
  2.4413938913910553],
 [4.3458254020379155,
  4.448157920197259,
  2.1995541990580083,
  10.986721820889377],
 [2.6945996854885004,
  1.509057856704581,
  3.8759681093830824,
  7.547929600539039],
 [2.958669540102495,
  4.37956969113504,
  4.393089578927343,
  0.00027040370592631654],
 [4.860390835134261,
  3.9374317459264283,
  2.5917595273059764,
  4.400679808239575],
 [4.011206357369818,
  3.190409389363397,
  3.387001291351266,
  0.07751330561061748],
 [4.582669168774147, 4.7008101675386, 4.9843915449059, 0.1842654556178927],
 [1.6362639200156806,
  2.0658243422421685,
  3.439729864510328,
  1.544319292082144],
 [4.043537120602787, 4.451214889948327, 2.291815056438512, 9.427522244942578],
 [3.915888778686337, 3.8438818438828912, 1.7242790047106258, 8.79648846853091],
 [2.102473514122666,
  1.9171218381656328,
  3.305508374670876,
  2.0263820276388946],
 [3.158221927478206,
  3.5938771139027192,
  2.2807787031067748,
  2.722746448854015],
 [4.105209678014413, 4.863807627997402, 1.5811797385283364, 22.11814283714212],
 [2.71159232169293, 2.485751698815974, 4.770404604595884, 7.076766381473809],
 [4.566823603712014, 1.173978946833493, 4.594323023206927, 26.713072039033],
 [4.129879735012713,
  2.6830943369832547,
  1.2799129099879867,
  4.065697515781985],
 [3.245776935779996,
  3.089041361947823,
  2.3831111814116324,
  0.8087460516982647],
 [3.775004755243238,
  2.3828772458145564,
  4.988532741347909,
  12.815085202369124],
 [4.634988140685998, 2.930148994098645, 4.4786927922999995, 5.557322727295995],
 [3.8179745320317577,
  1.8172505337286369,
  4.444738101812622,
  13.179058055621569],
 [2.0182488330302255,
  2.5496736738161694,
  1.5998678063260856,
  0.9103626066101844],
 [4.90309258681251, 3.4667865952856523, 4.721075464583739, 3.8568720822466336],
 [3.786324847416041, 2.40306628113669, 2.283604311043473, 0.02701762820625836],
 [1.9675656243749646,
  1.134535139179524,
  2.216609717607681,
  1.1518969249507105],
 [1.5385790041818797,
  2.800417641684175,
  3.7457335167036097,
  0.6874541031080226],
 [3.854967702577393,
  3.4077832998275293,
  1.1554156328745853,
  9.778434181504839],
 [3.6258340305960246,
  3.8456678990157758,
  2.3657860219805067,
  3.9703795800917923],
 [4.3288242853005485,
  3.671201650539603,
  1.0045982015136259,
  15.39064549045211],
 [4.348478749280432,
  1.733557002062422,
  1.7531134926782577,
  0.0008315516013458445],
 [1.5347104598472776,
  3.153562270401028,
  2.8662411119979474,
  0.06334782012174234],
 [2.832373750519524,
  4.3353254727750885,
  1.139012401196383,
  14.468356023815298],
 [2.9032529298796668, 4.555758694138406, 3.655449831258628, 1.17662461143561],
 [2.6602807432504774,
  4.41827859117587,
  4.229878196472022,
  0.04721294505362342],
 [3.42082874935801,
  2.938092786665137,
  2.6461084607053476,
  0.14582111514628554],
 [1.222967438632569,
  1.4888029553265216,
  1.4114458096160343,
  0.003659196841702431],
 [2.7316850082981223,
  4.665848132131451,
  4.595917898736861,
  0.0066792942910983795],
 [2.1490833657539556,
  2.7733136340136433,
  3.3932246362998226,
  0.41293524803497317],
 [3.5126075148769127,
  4.9707553667062285,
  4.8456288322114105,
  0.02749783258229099],
 [1.7036116512612671,
  3.5717354857267685,
  1.4869831846461476,
  3.70211179852432],
 [3.807738226385973,
  4.5032255859153745,
  2.7419599575434352,
  5.9059097741697615],
 [3.234653704658429, 3.317321706490886, 1.970379189990283, 2.9342419420303796],
 [1.016169735211558,
  2.3737488403695126,
  3.8995954527868273,
  1.1829271948186246],
 [3.1115583185472597,
  3.4165418651641235,
  4.073744689933028,
  0.6719652157434275],
 [2.2019255697616797,
  3.657392667709142,
  1.3207820464541657,
  6.0109806788761935],
 [4.737835388347947, 3.881728475314161, 3.7924596611389, 0.01887771839686493],
 [2.659782754602494, 2.5343127368465463, 4.9157723398410935, 7.54227925079901],
 [2.284667605551606,
  3.2596104100041865,
  1.0983910421354346,
  5.335691725299953],
 [2.445603589222424,
  2.625688086947879,
  2.4972297553537977,
  0.020178116340371244],
 [3.7616268283388363,
  4.309728605165561,
  1.2909471322729256,
  17.139930849578626],
 [2.679625969293971, 4.695393457710123, 4.133574857516475, 0.422898757409439],
 [2.7492345297373397,
  4.014635428369438,
  1.0110293765486045,
  12.401314905350047],
 [1.363482432479434, 4.713234856190606, 1.0429605806678022, 9.183676788071713],
 [4.620519432503656,
  1.3618668787394617,
  1.659673381532326,
  0.20489396117435182],
 [4.138931058520466, 1.6377028875003803, 2.850271065604626, 3.042779840294614],
 [3.678338957205359,
  1.2423933864362269,
  4.218116000470335,
  16.285707934493118],
 [2.2097144750629747,
  3.680042073014327,
  2.501709593362307,
  1.5340582919880528],
 [4.153510789516929, 3.6520298265147626, 1.858893495922922, 6.677470329979877],
 [1.341185053732071,
  1.3195961084024792,
  4.262170946011434,
  5.8064908122337675],
 [4.187063382718319, 4.7233153148388425, 2.727786997799847, 8.336722187618236],
 [2.7524493261322425,
  1.1411036768432599,
  4.083658326296682,
  11.916217216152011],
 [1.528150828917552,
  1.0765191796016103,
  2.1416493498188527,
  0.8668452994187462],
 [3.1472857258512597,
  1.0445695381485223,
  3.176204725273834,
  7.150426346811694],
 [1.5591987284356312,
  1.6936840019523456,
  3.144848401756421,
  1.6417412397759121],
 [1.8118232065254696,
  2.3213264623109535,
  3.139003643610357,
  0.6056888496703248],
 [4.629480897906726,
  4.306970670251901,
  3.995498493306813,
  0.22456435255701168],
 [1.1803653264547993,
  4.397998831575995,
  2.789487214775028,
  1.5269852829420845],
 [3.7106652820705683,
  3.0488707069310115,
  1.4083587376198414,
  4.993218742301966],
 [1.144545577560236, 3.830950618843174, 2.7127765078550823, 0.715520303352315],
 [3.2098933794590834,
  3.672149298815798,
  1.4722257120775026,
  7.767402375131977],
 [4.334363263688385,
  1.9468640837275522,
  4.2215258684602865,
  11.213184649904102],
 [1.7859786096492951,
  4.530222680221378,
  1.2669835849625075,
  9.509201457765421],
 [1.2538877054126338,
  4.08935338189937,
  4.262497271991879,
  0.018795028557219394],
 [1.2164300631144536,
  4.1452228499072294,
  2.1011826989486626,
  2.541183307727359],
 [1.3690738246936953,
  4.551671334602313,
  4.808185074105767,
  0.045042048666774956],
 [3.831893932346712, 1.114886649189021, 1.908997348209562, 1.2082187694476743],
 [4.373303656489842,
  1.8971939832560545,
  1.3146346718456043,
  0.7420957324054994],
 [4.215554508936311,
  1.8456582326867723,
  3.9569647710416533,
  9.395660136190838],
 [4.344891484237216, 2.1159173194765626, 4.143120830996502, 8.927783256842847],
 [3.409660042207056,
  1.2405913391974162,
  2.7447317734308307,
  3.857072983462115],
 [1.1275840924152334,
  2.7718931844626247,
  4.364964770556382,
  1.4308349110654137],
 [1.0588280014332878,
  3.7611617458603703,
  4.839482972849538,
  0.6155902480504866],
 [2.5784829060890244,
  4.288662259831485,
  3.1316501415735183,
  1.725877934522129],
 [1.2535404885074546,
  4.333581009369373,
  3.804887191102942,
  0.17519303455546567],
 [1.4391705423376138,
  1.524736406918307,
  1.7632864951160085,
  0.04094882347814548],
 [4.900163329121861, 4.468843447320165, 3.376867511211715, 2.921505418104924],
 [2.018692958323992, 3.525095743830818, 4.978721079692072, 2.132775976302815],
 [3.986694139690787,
  3.5744586880437472,
  1.3233962195790228,
  10.10085219906502],
 [1.5301016344222682,
  4.468296961707732,
  4.32342471519269,
  0.016056861924882457],
 [3.3751130191552074,
  4.126648104115262,
  3.1893961944731433,
  1.4824190676785498],
 [1.9905463635573981,
  2.5488304184899797,
  1.2587118633177314,
  1.6565385423282546],
 [2.552798961343791, 2.470322479160822, 2.88482644256247, 0.2193027077088335],
 [1.560843444958663, 1.8859929665674011, 4.051782908633044, 3.660682087950742],
 [4.518987144354405, 2.8352529551921846, 4.996659768474107, 10.55562960381974],
 [1.8902885513388736,
  1.090572372961509,
  3.9620696222245173,
  7.793183772156674],
 [2.837453401586004,
  2.6243710485171365,
  3.718016759907804,
  1.6968835942523173],
 [4.633952246723115,
  4.7009931806136285,
  4.80227486578186,
  0.023767494155873762],
 [3.0368149424743565,
  3.283323712664274,
  2.9979441907339135,
  0.1236613388506362],
 [1.8384420538166801,
  2.7427093441373893,
  2.6114288429344796,
  0.015842379130126913],
 [3.871317679113757,
  3.1449976100890655,
  2.0915751760919146,
  2.1479983387895807],
 [2.099998646950542,
  2.3143206094312383,
  1.7580331313368949,
  0.3249283368443198],
 [3.5186041717338967, 1.1565658066072246, 2.369275874087212, 2.58734524728803],
 [2.8023638189920934, 4.201212814764325, 2.476478751243142, 4.16810646096684],
 [3.1067542488832904,
  2.2317373407853602,
  3.0208265354118837,
  0.9672285297070201],
 [1.5779326155411493,
  4.397116705736808,
  4.28350480361812,
  0.01018371224710072],
 [4.949119817700403,
  3.6429821504948134,
  2.9493706592286615,
  1.1905031030324817],
 [2.4512461095175246,
  3.743197194945863,
  3.307334113432511,
  0.23283973247801043],
 [3.89451363517394, 1.0191518430824273, 4.586402038546808, 24.77937646850935],
 [1.7009994254213918,
  4.432217140583315,
  4.0989373364710815,
  0.09446961945785777],
 [3.429468231933705, 2.67669828514642, 2.114037681597649, 0.5428624520413814],
 [2.611445315757271,
  4.807912828788252,
  4.627437870751758,
  0.04252896751601671],
 [4.982876355849423,
  1.8463902932221927,
  1.915679754560066,
  0.01196146807128109],
 [1.3828532941159857,
  4.853980600047417,
  3.537718953190479,
  1.1979275886926017],
 [4.065161111109087,
  2.1173209958969883,
  1.5534942283555533,
  0.6461586265268637],
 [2.8406261517556914,
  1.4540963268691622,
  3.3558813748308465,
  5.136968972051234],
 [1.3173807789349596,
  2.4137478799601104,
  3.6151176385493677,
  0.9506807890582227],
 [4.98866058302202, 2.952147552580573, 3.356750070580415, 0.408329844520584],
 [3.275889237753974,
  3.9095072070009524,
  1.2399642553820338,
  11.672746105207528],
 [1.4454872635767289,
  2.657589824110316,
  2.7669981360903986,
  0.008651370448715287],
 [4.077997770937953,
  1.4213988023540534,
  3.3774291608741414,
  7.801321398422354],
 [4.418089046799263,
  3.9486056983485835,
  3.3745959448491334,
  0.727851888311662],
 [3.8205004202652733,
  1.0188661926762945,
  1.2208728935656237,
  0.07795102101162663],
 [3.1676086858411816,
  4.4517147185584065,
  3.0603430083393395,
  3.066110958299664],
 [2.7412712740688496,
  1.937121824867535,
  2.2855471121817788,
  0.16639541420174597],
 [3.803009405708048, 4.2014803794452, 3.588973740883934, 0.7133768372518275],
 [3.1854282001453966,
  3.345402573860775,
  2.7578401714800815,
  0.5498520145584768],
 [2.129580934718465, 2.0849750508691702, 4.928074797445353, 8.606931522335575],
 [3.582637585489866, 2.9125484631397835, 4.296085173460144, 3.428895552117278],
 [2.005248419443745, 3.130359290179915, 4.671303124393457, 2.3807391069171024],
 [4.536356735976824, 4.404165886472644, 4.83904103596097, 0.42894971762824363],
 [4.379298633663491,
  2.0498636957070704,
  2.290322332569241,
  0.12660630310548648],
 [2.4894375956837487,
  1.0687802436527583,
  4.782555646048527,
  17.16732065951714],
 [1.7771460406706723,
  1.9268565119409597,
  1.4745184949012335,
  0.1818106428219728],
 [1.7642941533489442,
  2.486803291169201,
  2.193363602545624,
  0.07595890676735433],
 [4.648234517022095, 2.210410128281588, 3.625612837512709, 4.654739043086355],
 [4.987155886111147,
  2.8388485947587876,
  1.8827336922659388,
  2.279518506917332],
 [1.6589824803402222, 1.676071538768832, 4.933668554305601, 8.802509874044022],
 [2.644822832864381,
  2.6833346215651206,
  2.4785861107724214,
  0.055438068812355384],
 [1.7136859953960144,
  2.9059002512731986,
  4.2158675305784055,
  1.470354813641605],
 [4.914802109669035, 4.305585785082451, 2.79837026988343, 5.5824745586533995],
 [3.2703363636167877,
  1.216107466561899,
  1.2778688132336318,
  0.006237290069768189],
 [4.696206040367107,
  3.2089841369028855,
  1.9598014749680805,
  3.664114552718608],
 [3.7536224895015033,
  4.100165797652629,
  4.248754420120971,
  0.04143732482302237],
 [2.119419438930699, 4.756883202548664, 3.043194443283673, 3.1120804381373133],
 [4.743384253143388,
  3.720405640841235,
  2.8273975084700327,
  1.8913379622399453],
 [3.200093437483908,
  1.8707058828523198,
  2.827896419048788,
  1.4659847604870122],
 [3.4965125145727285, 1.3909072240701774, 3.60542099274815, 8.57357321693519],
 [1.9970528454923224,
  1.539015924695534,
  1.5000304575960919,
  0.0015176270040447957],
 [2.002515825509517,
  3.6119142058372917,
  2.4730141145168676,
  1.2987250483642983],
 [2.7221475894401914,
  1.4130074712057956,
  2.8012511597308087,
  2.6230893719111132],
 [4.138402592195539,
  2.8077381494201274,
  2.1545451928102084,
  0.8828475739927203],
 [3.008425302044309, 2.838028356206219, 4.3404556052805265, 3.395440623220209],
 [3.9406764586503904,
  3.0427507102185087,
  3.040132378609634,
  1.3507969801038262e-05],
 [1.8656189128088032, 1.790209932911837, 4.233635704802488, 5.569179818005522],
 [1.2878153470863163,
  4.056171914154808,
  4.535767919807636,
  0.14810670341963406],
 [4.575223393792036, 3.3347821485958615, 1.2885707566407731, 9.57818684883466],
 [2.8257332727201927,
  2.4247765846843836,
  2.7230299607606767,
  0.12568165949693247],
 [2.2270006596366922,
  1.1913557511479351,
  4.195501150092936,
  10.049217521679845],
 [1.3313232055435944,
  1.8616021485186565,
  1.0351899042728023,
  0.45461838267209187],
 [4.548410858665534,
  2.9957410912450353,
  4.3440050622886055,
  4.134086415460299],
 [2.513593124544346,
  2.581549082055816,
  3.0785384261086524,
  0.31042675018937926],
 [3.990753124070536,
  2.1306688148510142,
  1.0860307776444578,
  2.1774918447495164],
 [3.424067400698733,
  4.938654411891039,
  1.7614597805478844,
  17.282236712337678],
 [4.29934427406726, 4.324177472532821, 3.1668892608779484, 2.8790902982777125],
 [3.6743547347930017,
  2.3592919828837315,
  2.444345057819267,
  0.013290188026632344],
 [4.342649753120276,
  2.3273238956828206,
  1.4156086442190596,
  1.8048588685819436],
 [2.123777509387738, 4.808605120909803, 1.8449866113634044, 9.326605748293192],
 [1.7295723966782708,
  2.579623813220033,
  3.329853601286825,
  0.4867403585517426],
 [1.9687020603943015,
  2.1878780171797745,
  3.9926118277583487,
  3.206094428875383],
 [2.7440480686893847,
  1.5920965667057332,
  2.842046239969557,
  2.1436149334096077],
 [1.1597713280944775,
  4.248489380099054,
  2.4045038694328995,
  1.9717751123105356],
 [4.727016223019934,
  3.9222714974355606,
  1.0609267134370715,
  19.35073571630212],
 [4.103594089169457, 2.741645058599168, 2.081784084682367, 0.8933862979140078],
 [4.818846862049859,
  4.910251032353065,
  2.4941927077051402,
  14.064618537833605],
 [4.136507140088131, 4.327106405470194, 2.4788723876125722, 7.065090047902346],
 [1.2155502331224972,
  3.219178288502854,
  3.0355308746894423,
  0.02049805003836853],
 [4.411586618996498,
  2.8251390078899736,
  2.7698748248052465,
  0.006736779370393002],
 [1.2041419126049595,
  2.8338365483537897,
  4.499849810929632,
  1.6711082613560397],
 [4.869198565516111, 4.318884814742127, 3.3457008302730715, 2.305777495554286],
 [4.111330010131941, 4.819486970902608, 1.114940618337513, 28.211255164783125],
 [4.665339246446299, 4.760691997928995, 2.8515726031598865, 8.501966966089737],
 [3.1276416856851856,
  4.804570817114312,
  3.809271469008311,
  1.5491534424148967],
 [3.5303881056710704,
  3.416464246299984,
  4.507493349544687,
  2.1011890395103396],
 [3.243371953850084,
  1.8307605415341266,
  3.2523711687827985,
  3.2773896965298563],
 [3.7035563990917595,
  4.896150870483837,
  1.5530190383461924,
  20.696455428411664],
 [3.369330648476493, 4.224332074737008, 2.363712380070957, 5.832152401272011],
 [2.911983734833672,
  2.1102747108355326,
  1.382742893212714,
  0.7706602018746154],
 [1.7257533404655927,
  1.7396012947430965,
  1.925004904880757,
  0.029660952937826432],
 [4.532929479893207, 4.917829827186975, 2.845817669927428, 9.730464342214306],
 [2.6750174347029083,
  4.790495523874714,
  1.2017229495179418,
  17.226160763019134],
 [2.4095917565249216,
  2.6518526781382845,
  1.8274633552248591,
  0.8188006709018926],
 [3.403167141669519, 2.6341292588641263, 1.42638243650639, 2.4820189371338275],
 [4.611752596589733, 2.2812677313753458, 4.148892527909364, 8.042948135413834],
 [2.01746286985032,
  4.439908829307878,
  4.500401677707119,
  0.003691336386892765],
 [3.2139937885743852,
  3.070311589163969,
  1.4759938617238326,
  4.0847434745069755],
 [2.783288071825717,
  2.5807729678536453,
  2.6720445786093454,
  0.011593100285181785],
 [4.346948844801675, 4.279967916675125, 2.8358654432476724, 4.532633011153217],
 [4.292922610363771,
  3.2233033367797947,
  3.496412160781049,
  0.16010117826696038],
 [1.5643084501196567,
  4.5390608923794495,
  4.035261220625804,
  0.19852177793678705],
 [2.108715300392387,
  4.646433311305262,
  4.302163000222352,
  0.12496462706954807],
 [1.0965465042508966,
  3.4801022401752997,
  1.473322639024163,
  2.207986504665756],
 [4.727777000692081, 3.579423640474183, 3.9173125400060296, 0.269882569727273],
 [1.279528975320082,
  2.860033792936706,
  3.3562015736382906,
  0.1574987996192875],
 [4.032882644491034,
  4.123626092814534,
  3.3347332938690695,
  1.2549359837428267],
 [3.321958693119673,
  1.7773234025793387,
  4.834330876095757,
  15.522341472812874],
 [1.0594928236974321,
  3.5007779790818305,
  3.552598821925785,
  0.0014225808835618074],
 [4.982584635229868,
  3.607067036301593,
  2.6359895314085153,
  2.3492675306210073],
 [4.2365178938735, 1.8370255617111972, 4.49854109744042, 15.005036650932494],
 [2.087869810995267,
  1.8793748363567562,
  2.8893559491973684,
  1.0648781692013332]])

In [37]:
NN_train(sample_data)

SimpleNet(
  (linear1): Linear(in_features=3, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=128, bias=True)
  (linear3): Linear(in_features=128, out_features=64, bias=True)
  (linear4): Linear(in_features=64, out_features=64, bias=True)
  (linear5): Linear(in_features=64, out_features=1, bias=True)
)

In [ ]:
/content/weights.h5